# Calculate GWP and create tables

The script calculates the GWP of hydrogen and methane using a steady-state perturbation approach. This build on Sand et al. 2023, and use the same method to calculate GWP for different sensitivity tests using the OsloCTM3 model.

**Sensitivity tests**:
- **Size of perturbation**
- **Location of pertubation**
- **Sensitivity of chemical background**


For all tests, a set of three simulations are used:
- **CTRL**: fixed emission of hydrogen and surface concentration of methane
- **pertH2**: as CTRL, but with surface H2 increased. Run to steady state.
- **10CH4**: as CTRL, but with surface CH4 increased by 10%. Run to steady state.

For 10CH4 a separate control simulations for the two first set of sensitivity tests are used.

GWP is the ratio of the absolute global warming potential (AGWP) for hydrogen to that for CO2. AGWP is defined as the time-integrated radiative forcing to a 1 kg pulse emission of that gas over a given time horizon, here 100 years. 

AGWP is equal to the steady-state radiative forcing (W m-2) divided by the delta flux (Tg-H2 yr-1) for a perturbed vs control. This delta flux between the perturbed (10H2 or 10CH4) and the control simulation (CTRL) includes chemical feedbacks. 

Based on the three set of simulations, we calculate the radiative forcing in the perturbed relative to the control. As the model is run with fixed surface concentration of methane, we need to have a separate perturbed methane run. We calculate the radiative forcing per Tg-CH4 (including the feedbacks, as feedbacks are included in the delta flux) for the methane perturbed run, and map the changes in the methane loss in the hydrogen perturbation with the results from the methane perturbed run. 


#### OUTLINE:
**PART I: Read model results**

**PART II: GWP calculations**

**PART III: Main results and tabels**

**Appendix with additional results**


This is the main notebook for the GWP calculations. Plotting are done in separate notebooks. 

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:,.4f}'.format(x) if abs(x)<0 else ('{:,.2f}'.format(x) if abs(x)<10 else ('{:,.1f}'.format(x) if abs(x)<100 else '{:,.0f}'.format(x))))

Input and output path:

In [2]:
path = r"./input/"
pathssp = r"./input_ssp/"
outputpath= r"./output/"

Constants:

In [3]:
#AGWP100_CO2 [mW yr m-2 Tg-1] Source: Table 7.SM.6 in IPCC AR6: 0.0895 pW m-2 yr kg-1 (p=10^-12) 
agwp100_CO2 = 0.0895

#CH4 tau_strat[yr] 
tau_strat = 120.0*2.0 #Multiplied by two for not double counting OH loss in stratosphere. 

#CH4 tau_soil [yr] 
tau_soil = 160.0

#Specific RF for CH4 [mW m-2 ppb-1] Etminan et al., 2016
spec_rf_ch4 = 0.44800

#The adjustment is –14% ± 15% IPCC AR6
spec_rf_ch4 = spec_rf_ch4*(1.0-0.14)
spec_rf_ch4

0.38528

# Part I: Read model results

In this part, model results are read from the input files.

## 1. Hydrogen budget

### 1.1 H2 burden [Tg]:

In [4]:
file = 'H2_burden.txt'
df_h2_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
#delta = df_h2_burden.loc['10H2']-df_h2_burden.loc['CTRL']
#delta.name = 'deltaH2'
#df_h2_burden_conc = df_h2_burden.append(delta)
df_h2_burden_conc = df_h2_burden['OSLOCTM3']
df_h2_burden_conc

Scenario
CTRL    195
10H2    NaN
10CH4   196
Name: OSLOCTM3, dtype: float64

In [5]:
file = 'OSLOCTM3-emi_burden-H2.csv'
df_h2_burden = pd.read_csv(path + file, sep=',',index_col=0,header=0)
#delta = df_h2_burden.loc['10H2']-df_h2_burden.loc['CTRL']
#delta.name = 'deltaH2'
df_h2_burden.loc['deltaH2'] = df_h2_burden.iloc[0]-df_h2_burden['CNTR'].iloc[0]
df_h2_burden
 

,CNTR,antro1,H2_avi
Scenario,,,
YR4,205,207,206
deltaH2,0.00,2.02,1.61


In [6]:
file = 'OSLOCTM3-emi_burden-H2.csv'
df_h2_burden_ssp = pd.read_csv(pathssp + file, sep=',',index_col=0,header=0)

ssps = ['SSP119','SSP434','SSP585']
df_h2_burden_ssp.loc['deltaH2'] = np.nan
for ssp in ssps:
    name = '10CH4_'+ssp
    df_h2_burden_ssp[name].loc['deltaH2'] = df_h2_burden_ssp[name].iloc[0]-df_h2_burden_ssp['CNTR_'+ssp].iloc[0]
    name = 'antro10_'+ssp
    df_h2_burden_ssp[name].loc['deltaH2'] = df_h2_burden_ssp[name].iloc[0]-df_h2_burden_ssp['CNTR_'+ssp].iloc[0]
    name = 'CNTR_'+ssp
    df_h2_burden_ssp[name].loc['deltaH2'] = df_h2_burden_ssp[name].iloc[0]-df_h2_burden_ssp['CNTR_'+ssp].iloc[0]

df_h2_burden_ssp
 


/tmp/ipykernel_2927/3946670128.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_h2_burden_ssp[name].loc['deltaH2'] = df_h2_burden_ssp[name].iloc[0]-df_h2_burden_ssp['CNTR_'+ssp].iloc[0]
/tmp/ipykernel_2927/3946670128.py:10: FutureWarni

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR13,188,235,245,181,225,234,203,247,256
deltaH2,7.15,9.96,11.1,0.00,0.00,0.00,21.8,22.4,22.0


### 1.2 H2 loss
Hydrogen loss happens through two main processes. The largest loss is through soil sink. Remaining hydrogen is lost through reactions with OH as it ascends through the atmosphere. 

#### H2 soil sink [Tg/yr]

In [7]:
file = 'H2_drydep.txt'
df_h2_drydep_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_drydep_conc = df_h2_drydep_conc['OSLOCTM3']
df_h2_drydep_conc

Scenario
CTRL    59.5
10H2    0.00
10CH4   59.5
Name: OSLOCTM3, dtype: float64

In [8]:
file = 'OSLOCTM3-emi_prod-loss-H2_all_YR4.csv'
df_h2_prodloss = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_h2_drydep = df_h2_prodloss.loc['Drydeposition']
df_h2_drydep

CNTR     58.0
antro1   58.7
H2_avi   58.5
Name: Drydeposition, dtype: float64

In [9]:
file = 'OSLOCTM3-emi_prod-loss-H2_all_YR13.csv'
df_h2_prodloss_ssp = pd.read_csv(pathssp + file, sep=',',index_col=0,header=0)
df_h2_drydep_ssp = df_h2_prodloss_ssp.loc['Drydeposition']
df_h2_drydep_ssp

10CH4_SSP119     53.4
10CH4_SSP434     66.1
10CH4_SSP585     68.9
CNTR_SSP119      51.5
CNTR_SSP434      63.5
CNTR_SSP585      66.0
antro10_SSP119   58.5
antro10_SSP434   70.7
antro10_SSP585   73.0
Name: Drydeposition, dtype: float64

#### H2 atmospheric loss [Tg/yr]

In [10]:
file = 'H2_atm_loss.txt'
df_h2_atmloss_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmloss_conc = df_h2_atmloss_conc['OSLOCTM3']
df_h2_atmloss_conc


Scenario
CTRL    27.9
10H2    0.00
10CH4   27.2
Name: OSLOCTM3, dtype: float64

In [11]:
df_h2_atmloss = df_h2_prodloss.loc['Total loss'] - df_h2_prodloss.loc['Drydeposition']
df_h2_atmloss

CNTR     29.2
antro1   29.4
H2_avi   29.4
dtype: float64

In [12]:
df_h2_atmloss_ssp = df_h2_prodloss_ssp.loc['Total loss'] - df_h2_prodloss_ssp.loc['Drydeposition']
df_h2_atmloss_ssp

10CH4_SSP119     25.2
10CH4_SSP434     28.6
10CH4_SSP585     31.4
CNTR_SSP119      24.9
CNTR_SSP434      28.3
CNTR_SSP585      31.0
antro10_SSP119   27.8
antro10_SSP434   31.0
antro10_SSP585   33.8
dtype: float64

#### H2 total loss [Tg/yr]:

In [13]:
df_h2_loss_conc = df_h2_atmloss_conc + df_h2_drydep_conc
df_h2_loss_conc

Scenario
CTRL    87.4
10H2    0.00
10CH4   86.7
Name: OSLOCTM3, dtype: float64

In [14]:
df_h2_loss = df_h2_atmloss+ df_h2_drydep
df_h2_loss

CNTR     87.2
antro1   88.1
H2_avi   87.9
dtype: float64

In [15]:
df_h2_loss_ssp = df_h2_atmloss_ssp+ df_h2_drydep_ssp
df_h2_loss_ssp

10CH4_SSP119     78.6
10CH4_SSP434     94.8
10CH4_SSP585      100
CNTR_SSP119      76.4
CNTR_SSP434      91.8
CNTR_SSP585      97.0
antro10_SSP119   86.4
antro10_SSP434    102
antro10_SSP585    107
dtype: float64

### 1.3 H2 production

#### H2 atm. prod [Tg/yr]

In [16]:
file = 'H2_atm_prod.txt'
df_h2_atmprod_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmprod_conc = df_h2_atmprod_conc['OSLOCTM3']
df_h2_atmprod_conc

Scenario
CTRL    55.9
10H2    0.00
10CH4   58.7
Name: OSLOCTM3, dtype: float64

In [17]:
file = 'OSLOCTM3-emi_emissions_H2.csv'
df_h2_emis = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_h2_emis

,CNTR,antro1,H2_avi
Total emission [Tg],32.2,33.2,33.2


In [18]:
file = 'OSLOCTM3-emi_emissions_H2.csv'
df_h2_emis_ssp = pd.read_csv(pathssp + file, sep=',',index_col=0,header=0)
df_h2_emis_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Total emission [Tg],32.2,32.2,32.2,32.2,32.2,32.2,42.3,42.3,42.3


In [19]:
df_h2_atmprod = df_h2_prodloss.loc['Total production'] - df_h2_emis.iloc[0]
df_h2_atmprod

CNTR     55.8
antro1   55.8
H2_avi   55.6
dtype: float64

In [20]:
df_h2_atmprod_ssp = df_h2_prodloss_ssp.loc['Total production'] - df_h2_emis_ssp.iloc[0]
df_h2_atmprod_ssp

10CH4_SSP119     47.2
10CH4_SSP434     63.4
10CH4_SSP585     69.0
CNTR_SSP119      45.0
CNTR_SSP434      60.4
CNTR_SSP585      65.6
antro10_SSP119   44.9
antro10_SSP434   60.3
antro10_SSP585   65.5
dtype: float64

### 1.4 Estimated H2 emissions (Total loss = Total prod + emission)

In [21]:
df_h2_estemis_conc = df_h2_atmloss_conc + df_h2_drydep_conc - df_h2_atmprod_conc
df_h2_estemis_conc

Scenario
CTRL    31.5
10H2    0.00
10CH4   28.0
Name: OSLOCTM3, dtype: float64

In [22]:
df_h2_estemis = df_h2_atmloss + df_h2_drydep - df_h2_atmprod
df_h2_estemis

CNTR     31.4
antro1   32.3
H2_avi   32.3
dtype: float64

In [23]:
df_h2_estemis_ssp = df_h2_atmloss_ssp + df_h2_drydep_ssp - df_h2_atmprod_ssp
df_h2_estemis_ssp

10CH4_SSP119     31.4
10CH4_SSP434     31.3
10CH4_SSP585     31.3
CNTR_SSP119      31.4
CNTR_SSP434      31.3
CNTR_SSP585      31.3
antro10_SSP119   41.4
antro10_SSP434   41.3
antro10_SSP585   41.3
dtype: float64

### 1.5 H2 surface concentration [ppb]

In [24]:
file = 'H2_surfconc.txt'
df_h2_surfconc_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
#delta = df_h2_surfconc_conc.loc['10H2']-df_h2_surfconc_conc.loc['CTRL']
#delta.name = 'deltaH2'
df_h2_surfconc_conc.loc['deltaH2'] = df_h2_surfconc_conc.loc['10H2']-df_h2_surfconc_conc.loc['CTRL']
#df_h2_surfconc_conc
#df_h2_surfconc_conc = df_h2_surfconc_conc.append(delta)
#df_h2_surfconc_conc = pd.concat([df_h2_surfconc_conc, delta])
df_h2_surfconc_conc = df_h2_surfconc_conc['OSLOCTM3']
df_h2_surfconc_conc

Scenario
CTRL      532
10H2      NaN
deltaH2   NaN
Name: OSLOCTM3, dtype: float64

In [25]:
file = 'OSLOCTM3-emi_surfconc-H2.csv'
df_h2_surfconc = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_h2_surfconc.loc['delta'] = df_h2_surfconc.iloc[0]-df_h2_surfconc['CNTR'].iloc[0]
df_h2_surfconc

,CNTR,antro1,H2_avi
YR4,559,565,564
delta,0.00,5.92,4.56


In [26]:
file = 'OSLOCTM3-emi_surfconc-H2.csv'
df_h2_surfconc_ssp = pd.read_csv(pathssp + file, sep=',',index_col=0,header=0)

ssps = ['SSP119','SSP434','SSP585']
df_h2_surfconc_ssp.loc['delta'] = np.nan
df_h2_surfconc_ssp
for ssp in ssps:
    name = '10CH4_'+ssp
    df_h2_surfconc_ssp[name].loc['delta'] = df_h2_surfconc_ssp[name].iloc[0]-df_h2_surfconc_ssp['CNTR_'+ssp].iloc[0]
    name = 'antro10_'+ssp
    df_h2_surfconc_ssp[name].loc['delta'] = df_h2_surfconc_ssp[name].iloc[0]-df_h2_surfconc_ssp['CNTR_'+ssp].iloc[0]
    name = 'CNTR_'+ssp
    df_h2_surfconc_ssp[name].loc['delta'] = df_h2_surfconc_ssp[name].iloc[0]-df_h2_surfconc_ssp['CNTR_'+ssp].iloc[0]

df_h2_surfconc_ssp

/tmp/ipykernel_2927/698958631.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_h2_surfconc_ssp[name].loc['delta'] = df_h2_surfconc_ssp[name].iloc[0]-df_h2_surfconc_ssp['CNTR_'+ssp].iloc[0]
/tmp/ipykernel_2927/698958631.py:11: FutureWar

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
YR13,514,640,668,495,613,638,558,678,702
delta,19.1,26.5,29.5,0.00,0.00,0.00,63.0,64.6,63.5


### 1.6 H2 lifetime [yr]
We calculate the lifetime as burden divided by loss.

#### H2 total lifetime [yr]

In [27]:
df_h2_lifetime_conc = df_h2_burden_conc/df_h2_loss_conc
df_h2_lifetime_conc


Scenario
CTRL    2.23
10H2     NaN
10CH4   2.26
Name: OSLOCTM3, dtype: float64

In [28]:
df_h2_lifetime = df_h2_burden.iloc[0]/df_h2_loss
df_h2_lifetime

CNTR     2.35
antro1   2.34
H2_avi   2.35
dtype: float64

In [29]:
df_h2_lifetime_ssp = df_h2_burden_ssp.iloc[0]/df_h2_loss_ssp
df_h2_lifetime_ssp

10CH4_SSP119     2.39
10CH4_SSP434     2.48
10CH4_SSP585     2.44
CNTR_SSP119      2.37
CNTR_SSP434      2.45
CNTR_SSP585      2.41
antro10_SSP119   2.35
antro10_SSP434   2.43
antro10_SSP585   2.40
dtype: float64

#### H2 atmospheric lifetime [yr]
The atmospheric lifetime is the burden divided by only the atmospheric loss. This is the lifetime of the fraction of hydrogen which is not dry deposited.

In [30]:
df_h2_atm_lifetime_conc = df_h2_burden_conc/df_h2_atmloss_conc
df_h2_atm_lifetime_conc

Scenario
CTRL    6.99
10H2     NaN
10CH4   7.20
Name: OSLOCTM3, dtype: float64

In [31]:
df_h2_atm_lifetime = df_h2_burden.iloc[0]/df_h2_atmloss
df_h2_atm_lifetime

CNTR     7.02
antro1   7.02
H2_avi   7.02
dtype: float64

In [32]:
df_h2_atm_lifetime_ssp = df_h2_burden_ssp.iloc[0]/df_h2_atmloss_ssp
df_h2_atm_lifetime_ssp

10CH4_SSP119     7.48
10CH4_SSP434     8.21
10CH4_SSP585     7.81
CNTR_SSP119      7.26
CNTR_SSP434      7.95
CNTR_SSP585      7.55
antro10_SSP119   7.29
antro10_SSP434   7.98
antro10_SSP585   7.57
dtype: float64

#### H2 soil sink lifetime [yr]

In [33]:
df_h2_soil_sink_lifetime_conc = df_h2_burden_conc/df_h2_drydep_conc
df_h2_soil_sink_lifetime_conc

Scenario
CTRL    3.28
10H2     NaN
10CH4   3.29
Name: OSLOCTM3, dtype: float64

In [34]:
df_h2_soil_sink_lifetime = df_h2_burden.iloc[0]/df_h2_drydep
df_h2_soil_sink_lifetime

CNTR     3.53
antro1   3.52
H2_avi   3.52
dtype: float64

In [35]:
df_h2_soil_sink_lifetime_ssp = df_h2_burden_ssp.iloc[0]/df_h2_drydep_ssp
df_h2_soil_sink_lifetime_ssp

10CH4_SSP119     3.52
10CH4_SSP434     3.55
10CH4_SSP585     3.56
CNTR_SSP119      3.51
CNTR_SSP434      3.54
CNTR_SSP585      3.55
antro10_SSP119   3.46
antro10_SSP434   3.50
antro10_SSP585   3.51
dtype: float64

### 1.7 H2 flux  [Tg/yr]

The hydrogen flux is calculated as the burden divided by the total hydrogen lifetime. 

The hydrogen lifetime is calculated as burden divided by the total loss (soil sink + atm.loss). 

The difference in calculated flux in the perturbed and control run is calculated. These numbers include feedbacks. 

For the GWP calculations, the radiative forcing in the steady state simulations are divided by these flux numbers.

In [36]:
df_h2_flux_conc = df_h2_burden_conc/df_h2_lifetime_conc
#Add delta flux 10H2:
delta = df_h2_flux_conc.loc['10H2']-df_h2_flux_conc.loc['CTRL']
#delta.name = 'deltaH2'
#df_h2_flux_conc = df_h2_flux_conc.append(delta)
df_h2_flux_conc['deltaH2'] = delta

#Add delta flux 10CH4:
delta = df_h2_flux_conc.loc['10CH4']-df_h2_flux_conc.loc['CTRL']
#delta.name = 'deltaCH4'
#df_h2_flux_conc = df_h2_flux_conc.append(delta)
df_h2_flux_conc['deltaCH4'] = delta
df_h2_flux_conc


Scenario
CTRL        87.4
10H2         NaN
10CH4       86.7
deltaH2      NaN
deltaCH4   -0.75
Name: OSLOCTM3, dtype: float64

In [37]:
df_h2_flux = df_h2_burden.iloc[0]/df_h2_lifetime
df_h2_flux.name = 'h2_flux'

delta = df_h2_flux-df_h2_flux['CNTR']
delta.name = 'deltaH2'

df_h2_flux =pd.concat([df_h2_flux,delta],axis=1)

df_h2_flux['deltaCH4'] = df_h2_flux_conc['deltaCH4']
df_h2_flux

,h2_flux,deltaH2,deltaCH4
CNTR,87.2,0.00,-0.75
antro1,88.1,0.95,-0.75
H2_avi,87.9,0.71,-0.75


In [38]:
df_h2_flux_ssp = df_h2_burden_ssp.iloc[0]/df_h2_lifetime_ssp
df_h2_flux_ssp.name = 'h2_flux'

df_h2_flux_ssp = df_h2_flux_ssp.to_frame()

df_h2_flux_ssp['deltaH2'] = np.nan
df_h2_flux_ssp['deltaCH4'] = np.nan

ssps = ['SSP119','SSP434','SSP585']

for ssp in ssps:
    name = '10CH4_'+ssp
    df_h2_flux_ssp['deltaH2'].loc[name] = df_h2_flux_ssp['h2_flux'].loc[name]-df_h2_flux_ssp['h2_flux'].loc['CNTR_'+ssp]
    name = 'antro10_'+ssp
    df_h2_flux_ssp['deltaH2'].loc[name] = df_h2_flux_ssp['h2_flux'].loc[name]-df_h2_flux_ssp['h2_flux'].loc['CNTR_'+ssp]
    name = 'CNTR_'+ssp
    df_h2_flux_ssp['deltaH2'].loc[name] = df_h2_flux_ssp['h2_flux'].loc[name]-df_h2_flux_ssp['h2_flux'].loc['CNTR_'+ssp]

df_h2_flux_ssp

/tmp/ipykernel_2927/2795338316.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_h2_flux_ssp['deltaH2'].loc[name] = df_h2_flux_ssp['h2_flux'].loc[name]-df_h2_flux_ssp['h2_flux'].loc['CNTR_'+ssp]
/tmp/ipykernel_2927/2795338316.py:15: Fu

,h2_flux,deltaH2,deltaCH4
10CH4_SSP119,78.6,2.13,NaN
10CH4_SSP434,94.8,2.98,NaN
10CH4_SSP585,100,3.36,NaN
CNTR_SSP119,76.4,0.00,NaN
CNTR_SSP434,91.8,0.00,NaN
CNTR_SSP585,97.0,0.00,NaN
antro10_SSP119,86.4,9.90,NaN
antro10_SSP434,102,9.90,NaN
antro10_SSP585,107,9.89,NaN


In the methane run, the hydrogen surface concentration is kept fixed. Enhancing methane would influence H2. The hydrogen concentration would have increased, but since we run with fixed concentration, there is a negative flux to compensate. So the increased flux in H2 due to methane is -1*deltaCH4.


For the SSP's, the methane perturbation is done on emission driven simulations, and multilpying with -1 is not necessary.

In [39]:
#Check Fluxes equal to the loss
df_h2_loss_conc

Scenario
CTRL    87.4
10H2    0.00
10CH4   86.7
Name: OSLOCTM3, dtype: float64

In [40]:
df_h2_loss

CNTR     87.2
antro1   88.1
H2_avi   87.9
dtype: float64

In [41]:
df_h2_loss_ssp

10CH4_SSP119     78.6
10CH4_SSP434     94.8
10CH4_SSP585      100
CNTR_SSP119      76.4
CNTR_SSP434      91.8
CNTR_SSP585      97.0
antro10_SSP119   86.4
antro10_SSP434    102
antro10_SSP585    107
dtype: float64

## 2. Methane results

### 2.1 CH4 burden [Tg]

In [42]:
file = 'CH4_burden.txt'
df_ch4_burden_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
#delta = df_ch4_burden_conc.loc['10CH4']-df_ch4_burden_conc.loc['CTRL']
#delta.name = 'deltaCH4'
#df_ch4_burden_conc = df_ch4_burden_conc.append(delta)

df_ch4_burden_conc.loc['deltaCH4'] = df_ch4_burden_conc.loc['10CH4']-df_ch4_burden_conc.loc['CTRL']
df_ch4_burden_conc = df_ch4_burden_conc['OSLOCTM3']
df_ch4_burden_conc

Scenario
CTRL       4,975
10H2         NaN
10CH4      5,474
deltaCH4     498
Name: OSLOCTM3, dtype: float64

In [43]:
file = 'OSLOCTM3-emi_burden-CH4.csv'
df_ch4_burden = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_ch4_burden.loc['deltaCH4'] = df_ch4_burden.iloc[0]-df_ch4_burden['CNTR'].iloc[0]
df_ch4_burden

,CNTR,antro1,H2_avi
Scenario,,,
YR4,"4,975","4,975","4,975"
deltaCH4,0.00,0.01,0.01


In [44]:
file = 'OSLOCTM3-emi_burden-CH4.csv'
df_ch4_burden_ssp = pd.read_csv(pathssp + file, sep=',',index_col=0,header=0)
ssps = ['SSP119','SSP434','SSP585']
df_ch4_burden_ssp.loc['deltaCH4'] = np.nan
for ssp in ssps:
    name = '10CH4_'+ssp
    df_ch4_burden_ssp[name].loc['deltaCH4'] = df_ch4_burden_ssp[name].iloc[0]-df_ch4_burden_ssp['CNTR_'+ssp].iloc[0]
    name = 'antro10_'+ssp
    df_ch4_burden_ssp[name].loc['deltaCH4'] = df_ch4_burden_ssp[name].iloc[0]-df_ch4_burden_ssp['CNTR_'+ssp].iloc[0]
    name = 'CNTR_'+ssp
    df_ch4_burden_ssp[name].loc['deltaCH4'] = df_ch4_burden_ssp[name].iloc[0]-df_ch4_burden_ssp['CNTR_'+ssp].iloc[0]


df_ch4_burden_ssp

/tmp/ipykernel_2927/3875004381.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_ch4_burden_ssp[name].loc['deltaCH4'] = df_ch4_burden_ssp[name].iloc[0]-df_ch4_burden_ssp['CNTR_'+ssp].iloc[0]
/tmp/ipykernel_2927/3875004381.py:9: FutureWa

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR13,"4,309","6,715","7,385","3,916","6,104","6,713","3,916","6,104","6,713"
deltaCH4,392,611,672,0.00,0.00,0.00,0.06,0.02,0.02


### 2.2 CH4 atmospheric loss [Tg/yr]

This is atmospheric loss due to OH.

In [45]:
file = 'CH4_loss.txt'
df_ch4_loss_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_ch4_loss_conc = df_ch4_loss_conc['OSLOCTM3']
df_ch4_loss_conc

Scenario
CTRL    674
10H2    NaN
10CH4   719
Name: OSLOCTM3, dtype: float64

In [46]:
file = 'OSLOCTM3-emi_prod-loss-CH4_all_YR4.csv'
df_ch4_prodloss = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_ch4_loss=df_ch4_prodloss.loc['CH4 + OH -> H2O + CH3']
df_ch4_loss

CNTR     673
antro1   673
H2_avi   673
Name: CH4 + OH -> H2O + CH3, dtype: float64

In [47]:
file = 'OSLOCTM3-emi_prod-loss-CH4_all_YR13.csv'
df_ch4_prodloss_ssp = pd.read_csv(pathssp + file, sep=',',index_col=0,header=0)
df_ch4_loss_ssp=df_ch4_prodloss_ssp.loc['CH4 + OH -> H2O + CH3']
df_ch4_loss_ssp

10CH4_SSP119     545
10CH4_SSP434     769
10CH4_SSP585     892
CNTR_SSP119      511
CNTR_SSP434      724
CNTR_SSP585      841
antro10_SSP119   508
antro10_SSP434   721
antro10_SSP585   837
Name: CH4 + OH -> H2O + CH3, dtype: float64

### 2.3 CH4 surface concentration [ppb]

In [48]:
file = 'CH4_surfconc.txt'
df_ch4_surfconc_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
#delta = df_ch4_surfconc_conc.loc['10CH4']-df_ch4_surfconc_conc.loc['CTRL']
#delta.name = 'deltaCH4'
#df_ch4_surfconc_conc = df_ch4_surfconc_conc.append(delta)
df_ch4_surfconc_conc.loc['deltaCH4'] = df_ch4_surfconc_conc.loc['10CH4']-df_ch4_surfconc_conc.loc['CTRL']

df_ch4_surfconc_conc = df_ch4_surfconc_conc['OSLOCTM3']
df_ch4_surfconc_conc

Scenario
CTRL       1,813
10CH4      1,994
deltaCH4     181
Name: OSLOCTM3, dtype: float64

In [49]:
file = 'OSLOCTM3-emi_surfconc-CH4.csv'
df_ch4_surfconc = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_ch4_surfconc.loc['delta'] = df_ch4_surfconc.iloc[0]-df_ch4_surfconc['CNTR'].iloc[0]
df_ch4_surfconc.loc['delta'] = df_ch4_surfconc_conc['deltaCH4']
df_ch4_surfconc

,CNTR,antro1,H2_avi
YR4,"1,813","1,813","1,813"
delta,181,181,181


In [50]:
file = 'OSLOCTM3-emi_surfconc-CH4.csv'
df_ch4_surfconc_ssp = pd.read_csv(pathssp + file, sep=',',index_col=0,header=0)

ssps = ['SSP119','SSP434','SSP585']
df_ch4_surfconc_ssp.loc['delta'] = np.nan
df_ch4_surfconc_ssp
for ssp in ssps:
    name = '10CH4_'+ssp
    df_ch4_surfconc_ssp[name].loc['delta'] = df_ch4_surfconc_ssp[name].iloc[0]-df_ch4_surfconc_ssp['CNTR_'+ssp].iloc[0]
    name = 'antro10_'+ssp
    df_ch4_surfconc_ssp[name].loc['delta'] = df_ch4_surfconc_ssp[name].iloc[0]-df_ch4_surfconc_ssp['CNTR_'+ssp].iloc[0]
    name = 'CNTR_'+ssp
    df_ch4_surfconc_ssp[name].loc['delta'] = df_ch4_surfconc_ssp[name].iloc[0]-df_ch4_surfconc_ssp['CNTR_'+ssp].iloc[0]

df_ch4_surfconc_ssp

/tmp/ipykernel_2927/2565989243.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_ch4_surfconc_ssp[name].loc['delta'] = df_ch4_surfconc_ssp[name].iloc[0]-df_ch4_surfconc_ssp['CNTR_'+ssp].iloc[0]
/tmp/ipykernel_2927/2565989243.py:11: Futu

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
YR13,"1,570","2,445","2,690","1,427","2,223","2,446","1,427","2,223","2,446"
delta,143,222,245,0.00,0.00,0.00,0.00,0.00,0.00


### 2.4 CH4 lifetime [yr]

#### Lifetime due to OH [yr]

In [51]:
df_ch4_lifetime_conc = df_ch4_burden_conc.drop('deltaCH4')/df_ch4_loss_conc
df_ch4_lifetime_conc

Scenario
CTRL    7.38
10H2     NaN
10CH4   7.62
Name: OSLOCTM3, dtype: float64

In [52]:
df_ch4_lifetime = df_ch4_burden.drop('deltaCH4')/df_ch4_loss
df_ch4_lifetime

,CNTR,antro1,H2_avi
Scenario,,,
YR4,7.39,7.40,7.40


In [53]:
df_ch4_lifetime_ssp = df_ch4_burden_ssp.drop('deltaCH4')/df_ch4_loss_ssp
df_ch4_lifetime_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR13,7.91,8.73,8.28,7.67,8.43,7.99,7.71,8.47,8.02


#### Total CH4 lifetime [yr]

Inverse lifetimes (mean loss frequencies) are additive [Forster et al.,2007; Prather, 2007]. Added lifetime due to stratospheric chemistry in addion to OH and soil sink.

In [54]:
df_ch4_tot_lifetime_conc = 1.0/(1.0/df_ch4_lifetime_conc + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime_conc

Scenario
CTRL    6.85
10H2     NaN
10CH4   7.06
Name: OSLOCTM3, dtype: float64

In [55]:
df_ch4_tot_lifetime = 1.0/(1.0/df_ch4_lifetime + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime

,CNTR,antro1,H2_avi
Scenario,,,
YR4,6.87,6.87,6.87


In [56]:
df_ch4_tot_lifetime_ssp = 1.0/(1.0/df_ch4_lifetime_ssp + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR13,7.31,8.00,7.62,7.10,7.75,7.37,7.14,7.78,7.40


#### Other methane loss term, dervided based on the lifetime and burden.

In [57]:
#Soil loss:
df_ch4_loss_soil_conc = df_ch4_burden_conc.drop('deltaCH4')/tau_soil
df_ch4_loss_soil_conc

Scenario
CTRL    31.1
10H2     NaN
10CH4   34.2
Name: OSLOCTM3, dtype: float64

In [58]:
df_ch4_loss_soil = df_ch4_burden.drop('deltaCH4')/tau_soil
df_ch4_loss_soil

,CNTR,antro1,H2_avi
Scenario,,,
YR4,31.1,31.1,31.1


In [59]:
df_ch4_loss_soil_ssp = df_ch4_burden_ssp.drop('deltaCH4')/tau_soil
df_ch4_loss_soil_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR13,26.9,42.0,46.2,24.5,38.2,42.0,24.5,38.2,42.0


In [60]:
#Strat chemistry loss (other than OH)
df_ch4_loss_other_strat_conc = df_ch4_burden_conc.drop('deltaCH4')/(tau_strat)
df_ch4_loss_other_strat_conc

Scenario
CTRL    20.7
10H2     NaN
10CH4   22.8
Name: OSLOCTM3, dtype: float64

In [61]:
df_ch4_loss_other_strat = df_ch4_burden.drop('deltaCH4')/(tau_strat)
df_ch4_loss_other_strat

,CNTR,antro1,H2_avi
Scenario,,,
YR4,20.7,20.7,20.7


In [62]:
df_ch4_loss_other_strat_ssp = df_ch4_burden_ssp.drop('deltaCH4')/(tau_strat)
df_ch4_loss_other_strat_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR13,18.0,28.0,30.8,16.3,25.4,28.0,16.3,25.4,28.0


### 2.5 CH4 flux  [Tg/yr]

The methane flux is calculated as the burden divided by the total methane lifetime.

The difference in calculated flux in the perturbed and control run is calculated. These numbers include feedbacks.

In [63]:
df_ch4_flux_conc = df_ch4_burden_conc.drop('deltaCH4')/df_ch4_tot_lifetime_conc

df_ch4_flux_conc['deltaCH4'] = df_ch4_flux_conc.loc['10CH4']-df_ch4_flux_conc.loc['CTRL']
df_ch4_flux_conc

Scenario
CTRL        726
10H2        NaN
10CH4       776
deltaCH4   49.7
Name: OSLOCTM3, dtype: float64

In [64]:
df_ch4_flux = df_ch4_burden.drop('deltaCH4')/df_ch4_tot_lifetime
df_ch4_flux.loc['deltaH2'] = df_ch4_flux.iloc[0]-df_ch4_flux['CNTR'].iloc[0]

#Add same ch4 flux in OsloCTM3 sensitivity tests as in OsloCTM3 conc
df_ch4_flux.loc['deltaCH4'] =  df_ch4_flux_conc['deltaCH4']
df_ch4_flux

,CNTR,antro1,H2_avi
Scenario,,,
YR4,725,724,724
deltaH2,0.00,-0.29,-0.23
deltaCH4,49.7,49.7,49.7


In [65]:
df_ch4_flux_ssp = df_ch4_burden_ssp.drop('deltaCH4')/df_ch4_tot_lifetime_ssp

df_ch4_flux_ssp.loc['deltaH2'] = np.nan
df_ch4_flux_ssp.loc['deltaCH4'] = np.nan

ssps = ['SSP119','SSP434','SSP585']
for ssp in ssps:
    name = '10CH4_'+ssp
    df_ch4_flux_ssp.loc['deltaCH4'][name] = df_ch4_flux_ssp.iloc[0][name]-df_ch4_flux_ssp.iloc[0]['CNTR_'+ssp]
    name = 'antro10_'+ssp
    df_ch4_flux_ssp.loc['deltaCH4'][name] = df_ch4_flux_ssp.iloc[0][name]-df_ch4_flux_ssp.iloc[0]['CNTR_'+ssp]
    name = 'CNTR_'+ssp
    df_ch4_flux_ssp.loc['deltaCH4'][name] = df_ch4_flux_ssp.iloc[0][name]-df_ch4_flux_ssp.iloc[0]['CNTR_'+ssp]


df_ch4_flux_ssp

/tmp/ipykernel_2927/20625344.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_ch4_flux_ssp.loc['deltaCH4'][name] = df_ch4_flux_ssp.iloc[0][name]-df_ch4_flux_ssp.iloc[0]['CNTR_'+ssp]
/tmp/ipykernel_2927/20625344.py:11: FutureWarning: Ch

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR13,589,839,969,552,787,910,549,784,907
deltaH2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
deltaCH4,37.8,52.1,58.6,0.00,0.00,0.00,-2.93,-2.93,-3.16


## 3. Ozone  RF

Ozone RF is calculated using a radiative kernel (Skeie et al 2020) and the modelled changes in ozone. 

In [66]:
file = 'ozone_rf.txt'
df_ozone_rf_conc = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_rf_conc = df_ozone_rf_conc['OSLOCTM3'] 
df_ozone_rf_conc 

Scenario
10H2     NaN
10CH4   40.8
Name: OSLOCTM3, dtype: float64

In [67]:
#ADD here for senstest
df_ozone_rf = pd.DataFrame(data=[],columns=df_ch4_flux.columns,index=df_ozone_rf_conc.index)
df_ozone_rf

,CNTR,antro1,H2_avi
Scenario,,,
10H2,NaN,NaN,NaN
10CH4,NaN,NaN,NaN


In [68]:
file = 'RF_NRFmethod_OsloCTM3_senstest_yr8_net_yearly.csv'
ozone_rf_orig = pd.read_csv(path+file,sep=';',index_col=0,header=0,skiprows=3)
ozone_rf_orig['NET adj.']


CNTR (mW m-2)           NaN
antro1-CNTR (mW m-2)   0.20
H2_avi-CNTR (mW m-2)   0.16
Name: NET adj., dtype: float64

In [69]:
for scen in df_ozone_rf.columns[1:]:
    df_ozone_rf.loc['10H2'][scen] = ozone_rf_orig['NET adj.'].loc[scen+'-CNTR (mW m-2)']
df_ozone_rf
    


/tmp/ipykernel_2927/1928313599.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_ozone_rf.loc['10H2'][scen] = ozone_rf_orig['NET adj.'].loc[scen+'-CNTR (mW m-2)']


,CNTR,antro1,H2_avi
Scenario,,,
10H2,NaN,0.20,0.16
10CH4,NaN,NaN,NaN


In [70]:
#In OsloCTM-emi use the same RF in the methane pertubation as in OsloCTM3
df_ozone_rf.loc['10CH4'] = df_ozone_rf_conc.loc['10CH4'] 
df_ozone_rf

,CNTR,antro1,H2_avi
Scenario,,,
10H2,NaN,0.20,0.16
10CH4,40.8,40.8,40.8


Read the ozone RF from SSP sensitivity tests

In [71]:
ssps = ['SSP119','SSP434','SSP585']

df_ozone_rf_ssp = pd.DataFrame(data=[],columns=ssps,index=['pertH2','10CH4'])

for ssp in ssps:
    file = 'RF_NRFmethod_OsloCTM3_ssp_senstest_yr26_'+ssp+'_net_yearly.csv'
    ozone_rf_orig = pd.read_csv(pathssp+file,sep=';',index_col=0,header=0,skiprows=3)
    ozone_rf_orig['NET adj.']
    df_ozone_rf_ssp[ssp].loc['pertH2'] = ozone_rf_orig['NET adj.'].loc['antro10_'+ssp+'-CNTR_'+ssp+' (mW m-2)']
    df_ozone_rf_ssp[ssp].loc['10CH4'] = ozone_rf_orig['NET adj.'].loc['10CH4_'+ssp+'-CNTR_'+ssp+' (mW m-2)']
df_ozone_rf_ssp

/tmp/ipykernel_2927/1442894704.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_ozone_rf_ssp[ssp].loc['pertH2'] = ozone_rf_orig['NET adj.'].loc['antro10_'+ssp+'-CNTR_'+ssp+' (mW m-2)']
/tmp/ipykernel_2927/1442894704.py:10: FutureWarnin

,SSP119,SSP434,SSP585
pertH2,2.20,1.81,1.78
10CH4,33.5,43.9,49.7


## 4. Stratospheric H2O RF [mW m-2]

Stratospheric H2O RF calculated offline.

In [72]:
file = 'H2O_rf.txt'
df_h2o_rf_conc = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_h2o_rf_conc = df_h2o_rf_conc['OSLOCTM3']
df_h2o_rf_conc

Scenario
10H2     NaN
10CH4   9.53
Name: OSLOCTM3, dtype: float64

In [73]:
#ADD here for senstest
df_h2o_rf = pd.DataFrame(data=[],columns=df_ch4_flux.columns,index=df_h2o_rf_conc.index)

In [74]:
file = 'strat_h2o_h2.csv'
df_h2o_rf_orig = pd.read_csv(path+file,sep='\t',index_col=0,header=0)
df_h2o_rf_orig

,an01,ant1,an10,a100,zepl,maud,nemo,epia,munc,usdd,maxd
NET,0.02,0.17,1.67,16.8,0.15,0.18,0.18,0.14,0.15,0.13,0.16


In [75]:
scen_dict_strat_h2o = {'an01':'antro01',
                       'ant1':'antro1',
                       'an10':'antro10',
                       'a100':'antro100',
                       'zepl':'zep',
                       'maud':'maud',
                       'nemo':'nemo',
                       'epia':'epia',
                       'munc':'munich',
                       'usdd':'usdrydep',
                       'maxd':'maxdep'}
#for scen in scen_dict_strat_h2o:
#    df_h2o_rf[scen_dict_strat_h2o[scen]].loc['10H2'] = df_h2o_rf_orig[scen].loc['NET']
df_h2o_rf

,CNTR,antro1,H2_avi
Scenario,,,
10H2,NaN,NaN,NaN
10CH4,NaN,NaN,NaN


In [76]:
#In OsloCTM-emi use the same RF in the methane pertubation as in OsloCTM3
df_h2o_rf.loc['10CH4'] = df_h2o_rf_conc.loc['10CH4'] 
df_h2o_rf

,CNTR,antro1,H2_avi
Scenario,,,
10H2,NaN,NaN,NaN
10CH4,9.53,9.53,9.53


In [77]:
ssps = ['SSP119','SSP434','SSP585']

df_h2o_rf_ssp = pd.read_csv(pathssp+'strat_h2o_ssp.csv',sep=',',index_col=0,header=0) 
df_h2o_rf_ssp

,SSP119,SSP434,SSP585
pertH2,1.73,1.74,1.70
10CH4,8.33,12.4,13.6



# Part II: GWP calculations:

The pulse integrated to infinity of the effects of a short lived climate forcer is equal to the change respones of its effects at steady state multiplied by the steady state lifetime of the short lived forcer(:cite:p:`Prather2002a` and :cite:p:`Prather2007a`). 

Prather 2002: prove that: (a) the steadystate pattern of impacts caused by specified emissions, multiplied by (b) the steady-state lifetime of the source gas for that emission pattern, is exactly equal to (c) the integral of all impacts - independent of the number and atmospheric residence times of secondary impacts. Therefore, the AGWP for hydrogen is identical whether calculating by integrating a pulse or by using the steady state changes per flux, given that the perturbation reaches steady state before 100 years. The longest-lived chemical mode here is keyed to methane, which has an e-folding lifetime of about 12 years. Our perturbed experiments are run to steate-state.


### Change in H2 surface conc. caused by 1 Tg H2/yr [ppb yr Tg-1]

This is not used for the GWP calculation. Only for the per flux table and for the feedback factor calulations.

In [78]:
df_ch4_flux

,CNTR,antro1,H2_avi
Scenario,,,
YR4,725,724,724
deltaH2,0.00,-0.29,-0.23
deltaCH4,49.7,49.7,49.7


In [79]:
df_ch4_flux_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR13,589,839,969,552,787,910,549,784,907
deltaH2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
deltaCH4,37.8,52.1,58.6,0.00,0.00,0.00,-2.93,-2.93,-3.16


In [80]:
df_surf_h2_per_h2_flux = df_h2_surfconc.loc['delta']/df_h2_flux['deltaH2']
df_surf_h2_per_h2_flux.name = 'surf_h2_per_h2_flux'
df_surf_h2_per_h2_flux

CNTR      NaN
antro1   6.26
H2_avi   6.38
Name: surf_h2_per_h2_flux, dtype: float64

In [81]:
df_h2_surfconc_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
YR13,514,640,668,495,613,638,558,678,702
delta,19.1,26.5,29.5,0.00,0.00,0.00,63.0,64.6,63.5


In [82]:
df_h2_flux_ssp

,h2_flux,deltaH2,deltaCH4
10CH4_SSP119,78.6,2.13,NaN
10CH4_SSP434,94.8,2.98,NaN
10CH4_SSP585,100,3.36,NaN
CNTR_SSP119,76.4,0.00,NaN
CNTR_SSP434,91.8,0.00,NaN
CNTR_SSP585,97.0,0.00,NaN
antro10_SSP119,86.4,9.90,NaN
antro10_SSP434,102,9.90,NaN
antro10_SSP585,107,9.89,NaN


In [83]:
df_surf_h2_per_h2_flux_ssp = df_h2_surfconc_ssp.loc['delta']/df_h2_flux_ssp['deltaH2']
df_surf_h2_per_h2_flux_ssp.name = 'surf_h2_per_h2_flux'
df_surf_h2_per_h2_flux_ssp

10CH4_SSP119     8.95
10CH4_SSP434     8.89
10CH4_SSP585     8.79
CNTR_SSP119       NaN
CNTR_SSP434       NaN
CNTR_SSP585       NaN
antro10_SSP119   6.36
antro10_SSP434   6.53
antro10_SSP585   6.42
Name: surf_h2_per_h2_flux, dtype: float64

### Change in CH4 flux caused by 1 TgH2 /yr (includes H2 feedback) [Tg CH4/Tg H2]:

The ch4_flux is multiplied by -1 (see above).

In [84]:
df_ch4_flux_per_h2_flux = -1.0*df_ch4_flux.loc['deltaH2']/df_h2_flux['deltaH2']
df_ch4_flux_per_h2_flux.name = 'ch4_flux_per_h2_flux'
df_ch4_flux_per_h2_flux

CNTR      NaN
antro1   0.31
H2_avi   0.32
Name: ch4_flux_per_h2_flux, dtype: float64

In [85]:
df_ch4_flux_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR13,589,839,969,552,787,910,549,784,907
deltaH2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
deltaCH4,37.8,52.1,58.6,0.00,0.00,0.00,-2.93,-2.93,-3.16


In [86]:
df_ch4_flux_ssp_v2 = pd.DataFrame(data=[],columns=['SSP119','SSP434','SSP585'],index=['deltaH2','deltaCH4'])

for scen in df_ch4_flux_ssp_v2:
        df_ch4_flux_ssp_v2[scen].loc['deltaH2'] = df_ch4_flux_ssp.loc['deltaCH4']['antro10_'+scen]
        df_ch4_flux_ssp_v2[scen].loc['deltaCH4'] = df_ch4_flux_ssp.loc['deltaCH4']['10CH4_'+scen]
df_ch4_flux_ssp_v2

/tmp/ipykernel_2927/1045193769.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_ch4_flux_ssp_v2[scen].loc['deltaH2'] = df_ch4_flux_ssp.loc['deltaCH4']['antro10_'+scen]
/tmp/ipykernel_2927/1045193769.py:5: FutureWarning: ChainedAssignme

,SSP119,SSP434,SSP585
deltaH2,-2.93,-2.93,-3.16
deltaCH4,37.8,52.1,58.6


In [87]:
df_h2_flux_ssp_v2 = pd.DataFrame(data=[],columns=['SSP119','SSP434','SSP585'],index=['deltaH2','deltaCH4'])

for scen in df_h2_flux_ssp_v2:
        df_h2_flux_ssp_v2[scen].loc['deltaH2'] = df_h2_flux_ssp['deltaH2'].loc['antro10_'+scen]
        df_h2_flux_ssp_v2[scen].loc['deltaCH4'] = df_h2_flux_ssp['deltaH2'].loc['10CH4_'+scen]
df_h2_flux_ssp_v2

/tmp/ipykernel_2927/1255993474.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_h2_flux_ssp_v2[scen].loc['deltaH2'] = df_h2_flux_ssp['deltaH2'].loc['antro10_'+scen]
/tmp/ipykernel_2927/1255993474.py:5: FutureWarning: ChainedAssignmentE

,SSP119,SSP434,SSP585
deltaH2,9.90,9.90,9.89
deltaCH4,2.13,2.98,3.36


In [88]:
df_ch4_flux_per_h2_flux_ssp = -1.0*df_ch4_flux_ssp_v2.loc['deltaH2']/df_h2_flux_ssp_v2.loc['deltaH2']
df_ch4_flux_per_h2_flux_ssp.name = 'ch4_flux_per_h2_flux_ssp'
df_ch4_flux_per_h2_flux_ssp

SSP119   0.30
SSP434   0.30
SSP585   0.32
Name: ch4_flux_per_h2_flux_ssp, dtype: object

### Change in CH4 surface conc. caused by 1 Tg/yr CH4 [ppb yr/Tg CH4]

In [89]:
df_surf_ch4_per_ch4_flux_conc =  df_ch4_surfconc_conc.loc['deltaCH4']/df_ch4_flux_conc.loc['deltaCH4']
df_surf_ch4_per_ch4_flux_conc


np.float64(3.6503026966623184)

In [90]:
df_surf_ch4_per_ch4_flux =  df_ch4_surfconc.loc['delta']/df_ch4_flux.loc['deltaCH4']
df_surf_ch4_per_ch4_flux.name = 'surf_ch4_per_ch4_flux'
df_surf_ch4_per_ch4_flux

CNTR     3.65
antro1   3.65
H2_avi   3.65
Name: surf_ch4_per_ch4_flux, dtype: float64

OsloCTM3-emi set equal to OsloCTM3 concentration driven.

In [91]:
df_surf_ch4_per_ch4_flux[:] = df_surf_ch4_per_ch4_flux_conc
df_surf_ch4_per_ch4_flux

CNTR     3.65
antro1   3.65
H2_avi   3.65
Name: surf_ch4_per_ch4_flux, dtype: float64

In [92]:
df_ch4_surfconc_ssp_v2 = pd.DataFrame(data=[],columns=['SSP119','SSP434','SSP585'],index=['delta'])
for scen in df_ch4_surfconc_ssp_v2.columns:
        df_ch4_surfconc_ssp_v2[scen].loc['delta'] = df_ch4_surfconc_ssp.loc['delta']['10CH4_'+scen]

df_ch4_surfconc_ssp_v2

/tmp/ipykernel_2927/934454039.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_ch4_surfconc_ssp_v2[scen].loc['delta'] = df_ch4_surfconc_ssp.loc['delta']['10CH4_'+scen]


,SSP119,SSP434,SSP585
delta,143,222,245


In [93]:
df_surf_ch4_per_ch4_flux_ssp =  df_ch4_surfconc_ssp_v2.loc['delta']/df_ch4_flux_ssp_v2.loc['deltaCH4']
df_surf_ch4_per_ch4_flux_ssp.name = 'surf_ch4_per_ch4_flux'
df_surf_ch4_per_ch4_flux_ssp

SSP119   3.78
SSP434   4.27
SSP585   4.17
Name: surf_ch4_per_ch4_flux, dtype: object

### Change in CH4 surface concentration per emission H2 [ppb yr /Tg H2]

In [94]:
df_surf_ch4_per_h2_flux = df_surf_ch4_per_ch4_flux*df_ch4_flux_per_h2_flux
df_surf_ch4_per_h2_flux.name = 'surf_ch4_per_h2_flux'
df_surf_ch4_per_h2_flux

CNTR      NaN
antro1   1.12
H2_avi   1.18
Name: surf_ch4_per_h2_flux, dtype: float64

In [95]:
df_surf_ch4_per_h2_flux_ssp = df_surf_ch4_per_ch4_flux_ssp*df_ch4_flux_per_h2_flux_ssp
df_surf_ch4_per_h2_flux_ssp.name = 'surf_ch4_per_h2_flux'
df_surf_ch4_per_h2_flux_ssp

SSP119   1.12
SSP434   1.26
SSP585   1.33
Name: surf_ch4_per_h2_flux, dtype: object

In [96]:
df_h2_flux

,h2_flux,deltaH2,deltaCH4
CNTR,87.2,0.00,-0.75
antro1,88.1,0.95,-0.75
H2_avi,87.9,0.71,-0.75


### Change in H2 flux caused by 1 TgCH4/yr [Tg H2/Tg CH4]

We multiply by -1 (see above)

In [97]:
df_h2_flux_per_ch4_flux = -1.0*df_h2_flux['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_h2_flux_per_ch4_flux.name = 'h2_flux_per_ch4_flux'
df_h2_flux_per_ch4_flux

CNTR     0.02
antro1   0.02
H2_avi   0.02
Name: h2_flux_per_ch4_flux, dtype: float64

In [98]:
df_h2_flux_ssp_v2 = pd.DataFrame(data=[],columns=['SSP119','SSP434','SSP585'],index=['deltaH2','deltaCH4'])

for scen in df_h2_flux_ssp_v2.columns:
        df_h2_flux_ssp_v2[scen].loc['deltaH2'] = df_h2_flux_ssp['deltaH2'].loc['antro10_'+scen]
        df_h2_flux_ssp_v2[scen].loc['deltaCH4'] = df_h2_flux_ssp['deltaH2'].loc['10CH4_'+scen]
df_h2_flux_ssp_v2

/tmp/ipykernel_2927/3000563033.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_h2_flux_ssp_v2[scen].loc['deltaH2'] = df_h2_flux_ssp['deltaH2'].loc['antro10_'+scen]
/tmp/ipykernel_2927/3000563033.py:5: FutureWarning: ChainedAssignmentE

,SSP119,SSP434,SSP585
deltaH2,9.90,9.90,9.89
deltaCH4,2.13,2.98,3.36


Here, we do not multiply by -1.0 as the methane perturbation is done on emission driven hydrogen simulations.

In [99]:
df_h2_flux_per_ch4_flux_ssp = df_h2_flux_ssp_v2.loc['deltaCH4']/df_ch4_flux_ssp_v2.loc['deltaCH4']
df_h2_flux_per_ch4_flux_ssp.name = 'h2_flux_per_ch4_flux'
df_h2_flux_per_ch4_flux_ssp

SSP119   0.06
SSP434   0.06
SSP585   0.06
Name: h2_flux_per_ch4_flux, dtype: object

### HYDROGEN AGWP100 CH4 [mW m-2 yr Tg-1]

agwp_ch4 = RF per flux H2

In [100]:
df_h2_agwp_ch4 = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_h2_agwp_ch4.name = 'h2_agwp_ch4'
df_h2_agwp_ch4

CNTR      NaN
antro1   0.43
H2_avi   0.46
Name: h2_agwp_ch4, dtype: float64

In [101]:
#agwp_ch4 = RF per flux H2 (For the per flux table)
df_ch4_rf_per_h2_flux = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_ch4_rf_per_h2_flux.name = 'ch4_rf_per_h2_flux'

df_ch4_rf_per_h2_flux

CNTR      NaN
antro1   0.43
H2_avi   0.46
Name: ch4_rf_per_h2_flux, dtype: float64

In [102]:
df_h2_agwp_ch4_ssp = df_surf_ch4_per_h2_flux_ssp*spec_rf_ch4
df_h2_agwp_ch4_ssp.name = 'h2_agwp_ch4'
df_h2_agwp_ch4_ssp

SSP119   0.43
SSP434   0.49
SSP585   0.51
Name: h2_agwp_ch4, dtype: object

In [103]:
#agwp_ch4 = RF per flux H2 (For the per flux table)
df_ch4_rf_per_h2_flux_ssp = df_surf_ch4_per_h2_flux_ssp*spec_rf_ch4
df_ch4_rf_per_h2_flux_ssp.name = 'ch4_rf_per_h2_flux'

df_ch4_rf_per_h2_flux_ssp

SSP119   0.43
SSP434   0.49
SSP585   0.51
Name: ch4_rf_per_h2_flux, dtype: object

### Initialize H2 GWP table

In [104]:
antmod = len(df_h2_agwp_ch4.index)
df_h2_gwp = pd.DataFrame(np.empty([5,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','O3 CH4ind','strat H2O CH4ind'])
df_h2_gwp

,CNTR,antro1,H2_avi
O3,NaN,NaN,NaN
CH4,NaN,NaN,NaN
strat H2O,NaN,NaN,NaN
O3 CH4ind,NaN,NaN,NaN
strat H2O CH4ind,NaN,NaN,NaN


In [105]:
antmod = len(df_h2_agwp_ch4_ssp.index)
df_h2_gwp_ssp = pd.DataFrame(np.empty([5,antmod])*np.nan,columns=df_h2_agwp_ch4_ssp.index,
                         index=['O3','CH4','strat H2O','O3 CH4ind','strat H2O CH4ind'])
df_h2_gwp_ssp

,SSP119,SSP434,SSP585
O3,NaN,NaN,NaN
CH4,NaN,NaN,NaN
strat H2O,NaN,NaN,NaN
O3 CH4ind,NaN,NaN,NaN
strat H2O CH4ind,NaN,NaN,NaN


### Add methane GWP

In [106]:
df_h2_gwp.loc['CH4'] = df_h2_agwp_ch4/agwp100_CO2
df_h2_gwp

,CNTR,antro1,H2_avi
O3,NaN,NaN,NaN
CH4,NaN,4.83,5.10
strat H2O,NaN,NaN,NaN
O3 CH4ind,NaN,NaN,NaN
strat H2O CH4ind,NaN,NaN,NaN


In [107]:
df_h2_gwp_ssp.loc['CH4'] = df_h2_agwp_ch4_ssp/agwp100_CO2
df_h2_gwp_ssp

,SSP119,SSP434,SSP585
O3,NaN,NaN,NaN
CH4,4.82,5.44,5.74
strat H2O,NaN,NaN,NaN
O3 CH4ind,NaN,NaN,NaN
strat H2O CH4ind,NaN,NaN,NaN


### HYDROGEN AGWP100 strat H2O [mW m-2 yr Tg-1]

In [108]:
df_h2_agwp_h2o = df_h2o_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_h2_agwp_h2o

CNTR     NaN
antro1   NaN
H2_avi   NaN
dtype: float64

In [109]:
#Add to the flux table
df_h2o_rf_per_h2_flux = df_h2o_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_h2o_rf_per_h2_flux.name= 'h2o_rf_per_h2_flux'


#Strat H2O RF per methane flux 
df_h2o_rf_per_ch4_flux = df_h2o_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_h2o_rf_per_ch4_flux.name = 'h2o_rf_per_ch4_flux'


In [110]:
df_h2_flux_ssp_v2

,SSP119,SSP434,SSP585
deltaH2,9.90,9.90,9.89
deltaCH4,2.13,2.98,3.36


In [111]:
df_h2_agwp_h2o_ssp = df_h2o_rf_ssp.loc['pertH2'].astype(float)/df_h2_flux_ssp_v2.loc['deltaH2']
df_h2_agwp_h2o_ssp = df_h2_agwp_h2o_ssp
#df_h2o_rf_ssp.loc['pertH2']
#df_h2_flux['deltaH2']
df_h2_agwp_h2o_ssp

SSP119   0.17
SSP434   0.18
SSP585   0.17
dtype: object

In [112]:
#Add to the flux table
df_h2o_rf_per_h2_flux_ssp = df_h2o_rf_ssp.loc['pertH2'].astype(float)/df_h2_flux_ssp_v2.loc['deltaH2']
df_h2o_rf_per_h2_flux_ssp.name= 'h2o_rf_per_h2_flux'


#Strat H2O RF per methane flux 
df_h2o_rf_per_ch4_flux_ssp = df_h2o_rf_ssp.loc['10CH4']/df_ch4_flux_ssp_v2.loc['deltaCH4']
df_h2o_rf_per_ch4_flux_ssp.name = 'h2o_rf_per_ch4_flux'


### Add stratospheric H2O GWP

In [113]:
df_h2_gwp.loc['strat H2O'] = df_h2_agwp_h2o/agwp100_CO2
df_h2_gwp

,CNTR,antro1,H2_avi
O3,NaN,NaN,NaN
CH4,NaN,4.83,5.10
strat H2O,NaN,NaN,NaN
O3 CH4ind,NaN,NaN,NaN
strat H2O CH4ind,NaN,NaN,NaN


In [114]:
df_h2_gwp_ssp.loc['strat H2O'] = df_h2_agwp_h2o_ssp/agwp100_CO2
df_h2_gwp_ssp

,SSP119,SSP434,SSP585
O3,NaN,NaN,NaN
CH4,4.82,5.44,5.74
strat H2O,1.95,1.96,1.92
O3 CH4ind,NaN,NaN,NaN
strat H2O CH4ind,NaN,NaN,NaN


### HYDROGEN AGWP100 O3 [mW m-2 yr Tg-1]

In [115]:
df_h2_agwp_o3 = df_ozone_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_h2_agwp_o3.name = 'h2_agwp_o3'

df_h2_agwp_o3

CNTR      NaN
antro1   0.22
H2_avi   0.22
Name: h2_agwp_o3, dtype: float64

In [116]:
df_h2_agwp_o3_ssp = df_ozone_rf_ssp.loc['pertH2'].astype(float)/df_h2_flux_ssp_v2.loc['deltaH2']
df_h2_agwp_o3_ssp.name = 'h2_agwp_o3'

df_h2_agwp_o3_ssp

SSP119   0.22
SSP434   0.18
SSP585   0.18
Name: h2_agwp_o3, dtype: object

In [117]:
#Similar. To be used in the table:
df_ozone_rf_per_h2_flux = df_ozone_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_ozone_rf_per_h2_flux.name= 'ozone_rf_per_h2_flux'

df_ozone_rf_per_h2_flux


CNTR      NaN
antro1   0.22
H2_avi   0.22
Name: ozone_rf_per_h2_flux, dtype: float64

In [118]:
#Similar. To be used in the table:
df_ozone_rf_per_h2_flux_ssp = df_ozone_rf_ssp.loc['pertH2'].astype(float)/df_h2_flux_ssp_v2.loc['deltaH2']
df_ozone_rf_per_h2_flux_ssp.name= 'ozone_rf_per_h2_flux'

df_ozone_rf_per_h2_flux_ssp

SSP119   0.22
SSP434   0.18
SSP585   0.18
Name: ozone_rf_per_h2_flux, dtype: object

In [119]:
#Ozone RF per methane flux (move to the methane part?)
df_ozone_rf_per_ch4_flux = df_ozone_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_ozone_rf_per_ch4_flux.name = 'ozone_rf_per_ch4_flux'
df_ozone_rf_per_ch4_flux

CNTR     0.82
antro1   0.82
H2_avi   0.82
Name: ozone_rf_per_ch4_flux, dtype: object

In [120]:
#Ozone RF per methane flux (move to the methane part?)
df_ozone_rf_per_ch4_flux_ssp = df_ozone_rf_ssp.loc['10CH4']/df_ch4_flux_ssp_v2.loc['deltaCH4']
df_ozone_rf_per_ch4_flux_ssp.name = 'ozone_rf_per_ch4_flux'
df_ozone_rf_per_ch4_flux_ssp

SSP119   0.89
SSP434   0.84
SSP585   0.85
Name: ozone_rf_per_ch4_flux, dtype: object

### Add Ozone GWP

In [121]:
df_h2_gwp.loc['O3'] = df_h2_agwp_o3/agwp100_CO2
df_h2_gwp

,CNTR,antro1,H2_avi
O3,NaN,2.41,2.51
CH4,NaN,4.83,5.10
strat H2O,NaN,NaN,NaN
O3 CH4ind,NaN,NaN,NaN
strat H2O CH4ind,NaN,NaN,NaN


In [122]:
df_h2_gwp_ssp.loc['O3'] = df_h2_agwp_o3_ssp/agwp100_CO2
df_h2_gwp_ssp

,SSP119,SSP434,SSP585
O3,2.48,2.05,2.01
CH4,4.82,5.44,5.74
strat H2O,1.95,1.96,1.92
O3 CH4ind,NaN,NaN,NaN
strat H2O CH4ind,NaN,NaN,NaN


### For the per flux table

In [123]:
df_ch4_flux


,CNTR,antro1,H2_avi
Scenario,,,
YR4,725,724,724
deltaH2,0.00,-0.29,-0.23
deltaCH4,49.7,49.7,49.7


## Methane induced GWP:

### HYDROGEN AGWP100 methane induced O3 [mW m-2 yr Tg-1]

In [124]:
df_surf_ch4_per_h2_flux

CNTR      NaN
antro1   1.12
H2_avi   1.18
Name: surf_ch4_per_h2_flux, dtype: float64

In [125]:
df_surf_ch4_per_h2_flux_ssp

SSP119   1.12
SSP434   1.26
SSP585   1.33
Name: surf_ch4_per_h2_flux, dtype: object

In [126]:
#Wm-2/ppbCH4*ppbCH4/TgH2yr-1 -> Wm-2/TgH2yr-1
df_h2_agwp_ch4ind_o3 = df_ozone_rf.loc['10CH4'].astype(float)/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_o3.name = 'h2_agwp_ch4ind_o3'
df_h2_agwp_ch4ind_o3

CNTR      NaN
antro1   0.25
H2_avi   0.27
Name: h2_agwp_ch4ind_o3, dtype: float64

In [127]:
df_surf_ch4_per_h2_flux_ssp

SSP119   1.12
SSP434   1.26
SSP585   1.33
Name: surf_ch4_per_h2_flux, dtype: object

In [128]:
#Wm-2/ppbCH4*ppbCH4/TgH2yr-1 -> Wm-2/TgH2yr-1
df_h2_agwp_ch4ind_o3_ssp = df_ozone_rf_ssp.loc['10CH4'].astype(float)/df_ch4_surfconc_ssp_v2.loc['delta']*df_surf_ch4_per_h2_flux_ssp
df_h2_agwp_ch4ind_o3_ssp.name = 'h2_agwp_ch4ind_o3'
df_h2_agwp_ch4ind_o3_ssp

SSP119   0.26
SSP434   0.25
SSP585   0.27
Name: h2_agwp_ch4ind_o3, dtype: object

### Add methane induced O3 GWP

In [129]:
df_h2_gwp.loc['O3 CH4ind'] = df_h2_agwp_ch4ind_o3/agwp100_CO2
df_h2_gwp

,CNTR,antro1,H2_avi
O3,NaN,2.41,2.51
CH4,NaN,4.83,5.10
strat H2O,NaN,NaN,NaN
O3 CH4ind,NaN,2.82,2.98
strat H2O CH4ind,NaN,NaN,NaN


In [130]:
df_h2_gwp_ssp.loc['O3 CH4ind'] = df_h2_agwp_ch4ind_o3_ssp/agwp100_CO2
df_h2_gwp_ssp

,SSP119,SSP434,SSP585
O3,2.48,2.05,2.01
CH4,4.82,5.44,5.74
strat H2O,1.95,1.96,1.92
O3 CH4ind,2.94,2.79,3.03
strat H2O CH4ind,NaN,NaN,NaN


### HYDROGEN AGWP100 methane induced strat H2O [mW m-2 yr Tg-1]

In [131]:
df_h2_agwp_ch4ind_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_h2o.name = 'h2_agwp_ch4ind_h2o'
df_h2_agwp_ch4ind_h2o

CNTR      NaN
antro1   0.06
H2_avi   0.06
Name: h2_agwp_ch4ind_h2o, dtype: object

In [132]:
df_h2_agwp_ch4ind_h2o_ssp = df_h2o_rf_ssp.loc['10CH4']/df_ch4_surfconc_ssp_v2.loc['delta']*df_surf_ch4_per_h2_flux_ssp
df_h2_agwp_ch4ind_h2o_ssp

SSP119   0.07
SSP434   0.07
SSP585   0.07
dtype: object

### Add methane induced strat H2O GWP

In [133]:
df_h2_gwp.loc['strat H2O CH4ind'] = df_h2_agwp_ch4ind_h2o/agwp100_CO2
df_h2_gwp

,CNTR,antro1,H2_avi
O3,NaN,2.41,2.51
CH4,NaN,4.83,5.10
strat H2O,NaN,NaN,NaN
O3 CH4ind,NaN,2.82,2.98
strat H2O CH4ind,NaN,0.66,0.70


In [134]:
df_h2_gwp_ssp.loc['strat H2O CH4ind'] = df_h2_agwp_ch4ind_h2o_ssp/agwp100_CO2
df_h2_gwp_ssp

,SSP119,SSP434,SSP585
O3,2.48,2.05,2.01
CH4,4.82,5.44,5.74
strat H2O,1.95,1.96,1.92
O3 CH4ind,2.94,2.79,3.03
strat H2O CH4ind,0.73,0.79,0.83


# Methane GWP

Initialize CH4 GWP

In [135]:
antmod = len(df_h2_agwp_ch4.index)
df_ch4_gwp = pd.DataFrame(np.empty([3,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O'])
df_ch4_gwp

,CNTR,antro1,H2_avi
O3,NaN,NaN,NaN
CH4,NaN,NaN,NaN
strat H2O,NaN,NaN,NaN


In [136]:
antmod = len(df_h2_agwp_ch4_ssp.index)
df_ch4_gwp_ssp = pd.DataFrame(np.empty([3,antmod])*np.nan,columns=df_h2_agwp_ch4_ssp.index,
                         index=['O3','CH4','strat H2O'])
df_ch4_gwp_ssp

,SSP119,SSP434,SSP585
O3,NaN,NaN,NaN
CH4,NaN,NaN,NaN
strat H2O,NaN,NaN,NaN


### Methane AGWP100 O3 [mW m-2 yr Tg-1]

In [137]:
df_ch4_agwp_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_o3.name = 'ch4_agwp_o3'
df_ch4_agwp_o3

CNTR     0.82
antro1   0.82
H2_avi   0.82
Name: ch4_agwp_o3, dtype: object

In [138]:
df_ch4_agwp_o3_ssp = df_ozone_rf_ssp.loc['10CH4']/df_ch4_surfconc_ssp_v2.loc['delta']*df_surf_ch4_per_ch4_flux_ssp
df_ch4_agwp_o3_ssp.name = 'ch4_agwp_o3'
df_ch4_agwp_o3_ssp

SSP119   0.89
SSP434   0.84
SSP585   0.85
Name: ch4_agwp_o3, dtype: object

In [139]:
test = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_ch4_flux
test

CNTR     0.82
antro1   0.82
H2_avi   0.82
dtype: object

In [140]:
test2 = df_ozone_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
#test2 is equal test
test2

CNTR     0.82
antro1   0.82
H2_avi   0.82
dtype: object

In [141]:
test2 = df_ozone_rf_ssp.loc['10CH4']/df_ch4_flux_ssp_v2.loc['deltaCH4']
#test2 is equal test
test2

SSP119   0.89
SSP434   0.84
SSP585   0.85
dtype: object

### Add ozone GWP

In [142]:
df_ch4_gwp.loc['O3'] =df_ch4_agwp_o3/agwp100_CO2 

In [143]:
df_ch4_gwp_ssp.loc['O3'] =df_ch4_agwp_o3_ssp/agwp100_CO2 

### Methane AGWP100 Methane [mW m-2 yr Tg-1]

In [144]:
df_ch4_agwp =df_surf_ch4_per_ch4_flux*spec_rf_ch4
df_ch4_agwp.name = 'ch4_agwp'


In [145]:
df_ch4_agwp_ssp =df_surf_ch4_per_ch4_flux_ssp*spec_rf_ch4
df_ch4_agwp_ssp.name = 'ch4_agwp'

### Add methane GWP

In [146]:
#Add Methane GWP:
df_ch4_gwp.loc['CH4'] =df_ch4_agwp/agwp100_CO2 

In [147]:
df_ch4_gwp_ssp.loc['CH4'] =df_ch4_agwp_ssp/agwp100_CO2 

### Methane AGWP100 strat H2O [mW m-2 yr Tg-1]

In [148]:
df_ch4_agwp_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_h2o.name = 'ch4_agwp_h2o'

In [149]:
df_ch4_agwp_h2o_ssp = df_h2o_rf_ssp.loc['10CH4']/df_ch4_surfconc_ssp_v2.loc['delta']*df_surf_ch4_per_ch4_flux_ssp
df_ch4_agwp_h2o_ssp 

SSP119   0.22
SSP434   0.24
SSP585   0.23
dtype: object

### Add Strat H2O GWP:

In [150]:
df_ch4_gwp.loc['strat H2O'] = df_ch4_agwp_h2o/agwp100_CO2

In [151]:
df_ch4_gwp_ssp.loc['strat H2O'] = df_ch4_agwp_h2o_ssp/agwp100_CO2

# Part III: Main results and tables

## H2 GWP 100

In [152]:
df_h2_gwp = pd.concat([df_h2_gwp,df_h2_gwp_ssp],axis=1, join='inner',sort=False)
df_h2_gwp                  

,CNTR,antro1,H2_avi,SSP119,SSP434,SSP585
O3,NaN,2.41,2.51,2.48,2.05,2.01
CH4,NaN,4.83,5.10,4.82,5.44,5.74
strat H2O,NaN,NaN,NaN,1.95,1.96,1.92
O3 CH4ind,NaN,2.82,2.98,2.94,2.79,3.03
strat H2O CH4ind,NaN,0.66,0.70,0.73,0.79,0.83


In [153]:
df_h2_gwp = df_h2_gwp.drop('CNTR',axis=1)
df_h2_gwp.loc['total']=df_h2_gwp.sum()

df_h2_gwp_table = df_h2_gwp.copy()

df_h2_gwp_table.T


,O3,CH4,strat H2O,O3 CH4ind,strat H2O CH4ind,total
antro1,2.41,4.83,NaN,2.82,0.66,10.7
H2_avi,2.51,5.10,NaN,2.98,0.70,11.3
SSP119,2.48,4.82,1.95,2.94,0.73,12.9
SSP434,2.05,5.44,1.96,2.79,0.79,13.0
SSP585,2.01,5.74,1.92,3.03,0.83,13.5


## CH4 GWP 100

In [154]:
df_ch4_gwp = pd.concat([df_ch4_gwp,df_ch4_gwp_ssp],axis=1, join='inner',sort=False)
df_ch4_gwp

,CNTR,antro1,H2_avi,SSP119,SSP434,SSP585
O3,9.17,9.17,9.17,9.92,9.42,9.48
CH4,15.7,15.7,15.7,16.3,18.4,18.0
strat H2O,2.14,2.14,2.14,2.46,2.66,2.59


In [155]:
df_ch4_gwp = df_ch4_gwp.drop('CNTR',axis=1)
df_ch4_gwp.to_csv(outputpath + 'table_ch4_gwp.csv')
df_ch4_gwp.loc['total']=df_ch4_gwp.sum()
df_ch4_gwp.T

,O3,CH4,strat H2O,total
antro1,9.17,15.7,2.14,27.0
H2_avi,9.17,15.7,2.14,27.0
SSP119,9.92,16.3,2.46,28.6
SSP434,9.42,18.4,2.66,30.5
SSP585,9.48,18.0,2.59,30.0


## Table per flux H2

In [156]:
df_per_flux_h2 = pd.concat([df_h2_flux['deltaH2'],
                            df_surf_h2_per_h2_flux,
                            df_surf_ch4_per_h2_flux,
                            df_ch4_flux_per_h2_flux,
                            df_ch4_rf_per_h2_flux,
                            #df_trop_du_ozone_per_h2_flux*1000.,
                            #df_strat_du_ozone_per_h2_flux*1000.,
                            df_ozone_rf_per_h2_flux,
                            df_h2o_rf_per_h2_flux 
                            #df_aerosol_rf_per_h2_flux
                           ],axis=1, sort=False)

df_per_flux_h2

,deltaH2,surf_h2_per_h2_flux,surf_ch4_per_h2_flux,ch4_flux_per_h2_flux,ch4_rf_per_h2_flux,ozone_rf_per_h2_flux,h2o_rf_per_h2_flux
CNTR,0.00,NaN,NaN,NaN,NaN,NaN,NaN
antro1,0.95,6.26,1.12,0.31,0.43,0.22,NaN
H2_avi,0.71,6.38,1.18,0.32,0.46,0.22,NaN


In [157]:
df_h2_flux_per_ch4_flux

CNTR     0.02
antro1   0.02
H2_avi   0.02
Name: h2_flux_per_ch4_flux, dtype: float64

In [158]:
df_per_flux_h2.loc['SSP119'] = None
df_per_flux_h2.loc['SSP434'] = None
df_per_flux_h2.loc['SSP585'] = None

scen_list = ['SSP119','SSP434','SSP585']
df_h2_flux_ssp_v2
for scen in scen_list:
    df_per_flux_h2['deltaH2'].loc[scen] = df_h2_flux_ssp_v2.loc['deltaH2'][scen]
    df_per_flux_h2['surf_h2_per_h2_flux'].loc[scen] = df_surf_h2_per_h2_flux_ssp['antro10_'+scen]
    df_per_flux_h2['surf_ch4_per_h2_flux'].loc[scen] = df_surf_ch4_per_h2_flux_ssp[scen]
    df_per_flux_h2['ch4_flux_per_h2_flux'].loc[scen] = df_ch4_flux_per_h2_flux_ssp[scen]
    df_per_flux_h2['ch4_rf_per_h2_flux'].loc[scen] = df_ch4_rf_per_h2_flux_ssp[scen]
    df_per_flux_h2['ozone_rf_per_h2_flux'].loc[scen] = df_ozone_rf_per_h2_flux_ssp[scen]
    df_per_flux_h2['h2o_rf_per_h2_flux'].loc[scen] = df_h2o_rf_per_h2_flux_ssp[scen]
df_per_flux_h2

/tmp/ipykernel_2927/3254411175.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_per_flux_h2['deltaH2'].loc[scen] = df_h2_flux_ssp_v2.loc['deltaH2'][scen]
/tmp/ipykernel_2927/3254411175.py:9: FutureWarning: ChainedAssignmentError: behav

,deltaH2,surf_h2_per_h2_flux,surf_ch4_per_h2_flux,ch4_flux_per_h2_flux,ch4_rf_per_h2_flux,ozone_rf_per_h2_flux,h2o_rf_per_h2_flux
CNTR,0.00,NaN,NaN,NaN,NaN,NaN,NaN
antro1,0.95,6.26,1.12,0.31,0.43,0.22,NaN
H2_avi,0.71,6.38,1.18,0.32,0.46,0.22,NaN
SSP119,9.90,6.36,1.12,0.30,0.43,0.22,0.17
SSP434,9.90,6.53,1.26,0.30,0.49,0.18,0.18
SSP585,9.89,6.42,1.33,0.32,0.51,0.18,0.17


In [159]:
#Save to file:
df_per_flux_h2 = df_per_flux_h2.sort_index()
df_per_flux_h2.to_csv(outputpath + 'table_per_flux_h2.csv')

#Rename the columns:
columns_names={'deltaH2':'Flux H2 [Tg/yr]',
               'surf_h2_per_h2_flux': 'Surf. conc. H2 per flux [ppb yr/Tg]',
               'surf_ch4_per_h2_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'ch4_flux_per_h2_flux':'Flux CH4/Flux H2 [Tg CH4/Tg H2]',
               'ch4_rf_per_h2_flux':'CH4 RF per flux [mW m-2 yr/ Tg]',
               'trop_du_ozone_per_h2_flux':'Trop. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'strat_du_ozone_per_h2_flux':'Strat. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'ozone_rf_per_h2_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_h2_flux':'Strat. H2O RF per flux [mW m-2 yr/ Tg]',
               'aerosol_rf_per_h2_flux':'Aerosol RF per flux [mW m-2 yr/ Tg]'}

#Rename column names:
df_per_flux_h2.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])

df_per_flux_h2 = df_per_flux_h2.drop('CNTR')

sorted_list = ['antro1', 
               'H2_avi',
               'SSP119', 
               'SSP434', 
               'SSP585'] 
senslist = sorted_list
senslist = [sub.replace('antro', 'anthro') for sub in senslist]
df_per_flux_h2 = df_per_flux_h2.reindex(sorted_list)
df_per_flux_h2.index = [sub.replace('antro', 'anthro') for sub in df_per_flux_h2.index]


df_per_flux_h2.to_csv(outputpath + 'table_per_flux_h2_to_manuscript.csv')
df_per_flux_h2

,Flux H2 [Tg/yr],Surf. conc. H2 per flux [ppb yr/Tg],Surf. conc. CH4 per flux [ppb yr/Tg],Flux CH4/Flux H2 [Tg CH4/Tg H2],CH4 RF per flux [mW m-2 yr/ Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat. H2O RF per flux [mW m-2 yr/ Tg]
anthro1,0.95,6.26,1.12,0.31,0.43,0.22,NaN
H2_avi,0.71,6.38,1.18,0.32,0.46,0.22,NaN
SSP119,9.90,6.36,1.12,0.30,0.43,0.22,0.17
SSP434,9.90,6.53,1.26,0.30,0.49,0.18,0.18
SSP585,9.89,6.42,1.33,0.32,0.51,0.18,0.17


## Table per flux CH4

In [160]:
df_per_flux_ch4 = pd.concat([df_ch4_flux.loc['deltaCH4'],
                            df_surf_ch4_per_ch4_flux,
                            df_h2_flux_per_ch4_flux,
                            #df_trop_du_ozone_per_ch4_flux*1000.,
                            #df_strat_du_ozone_per_ch4_flux*1000.,
                            df_ozone_rf_per_ch4_flux,
                            df_h2o_rf_per_ch4_flux
                            #df_aerosol_rf_per_ch4_flux
                            ],axis=1,sort=False)


df_per_flux_ch4        

,deltaCH4,surf_ch4_per_ch4_flux,h2_flux_per_ch4_flux,ozone_rf_per_ch4_flux,h2o_rf_per_ch4_flux
CNTR,49.7,3.65,0.02,0.82,0.19
antro1,49.7,3.65,0.02,0.82,0.19
H2_avi,49.7,3.65,0.02,0.82,0.19


In [161]:
df_per_flux_ch4.loc['SSP119'] = None
df_per_flux_ch4.loc['SSP434'] = None
df_per_flux_ch4.loc['SSP585'] = None

In [162]:
df_h2o_rf_per_ch4_flux_ssp

SSP119   0.22
SSP434   0.24
SSP585   0.23
Name: h2o_rf_per_ch4_flux, dtype: object

In [163]:
scen_list = ['SSP119','SSP434','SSP585']

for scen in scen_list:
    df_per_flux_ch4['deltaCH4'].loc[scen] = df_ch4_flux_ssp_v2.loc['deltaCH4'][scen]
    df_per_flux_ch4['surf_ch4_per_ch4_flux'].loc[scen] = df_surf_ch4_per_ch4_flux_ssp[scen]
    df_per_flux_ch4['h2_flux_per_ch4_flux'].loc[scen] =df_h2_flux_per_ch4_flux_ssp[scen]
    df_per_flux_ch4['ozone_rf_per_ch4_flux'].loc[scen] = df_ozone_rf_per_ch4_flux_ssp[scen]
    df_per_flux_ch4['h2o_rf_per_ch4_flux'].loc[scen] = df_h2o_rf_per_ch4_flux_ssp[scen]
df_per_flux_ch4

/tmp/ipykernel_2927/113259963.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_per_flux_ch4['deltaCH4'].loc[scen] = df_ch4_flux_ssp_v2.loc['deltaCH4'][scen]
/tmp/ipykernel_2927/113259963.py:4: SettingWithCopyWarning: 
A value is trying

,deltaCH4,surf_ch4_per_ch4_flux,h2_flux_per_ch4_flux,ozone_rf_per_ch4_flux,h2o_rf_per_ch4_flux
CNTR,49.7,3.65,0.02,0.82,0.19
antro1,49.7,3.65,0.02,0.82,0.19
H2_avi,49.7,3.65,0.02,0.82,0.19
SSP119,37.8,3.78,0.06,0.89,0.22
SSP434,52.1,4.27,0.06,0.84,0.24
SSP585,58.6,4.17,0.06,0.85,0.23


In [164]:
#Save to file:
df_per_flux_ch4.to_csv(outputpath + 'table_per_flux_ch4.csv')

#Rename the columns:
columns_names={'deltaCH4':'Flux CH4 [Tg/yr]',
               'surf_ch4_per_ch4_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'h2_flux_per_ch4_flux':'Flux H2/Flux CH4 [Tg H2/Tg CH4]',
               'trop_du_ozone_per_ch4_flux':'Trop. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'strat_du_ozone_per_ch4_flux':'Strat. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'ozone_rf_per_ch4_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_ch4_flux':'Strat H2O RF per flux [mW m-2 yr/ Tg]',
               'aerosol_rf_per_ch4_flux':'Aerosol RF per flux [mW m-2 yr/ Tg]'}
               
#Rename column names:

df_per_flux_ch4.rename(columns=dict(columns_names),inplace=True)
df_per_flux_ch4 = df_per_flux_ch4.drop('CNTR')

sorted_list = ['antro1', 
               'H2_avi',
               'SSP119', 
               'SSP434', 
               'SSP585'] 
senslist = sorted_list
senslist = [sub.replace('antro', 'anthro') for sub in senslist]
df_per_flux_ch4 = df_per_flux_ch4.reindex(sorted_list)
df_per_flux_ch4.index = [sub.replace('antro', 'anthro') for sub in df_per_flux_ch4.index]
df_per_flux_ch4.to_csv(outputpath + 'table_per_flux_ch4_to_manuscript.csv')
df_per_flux_ch4

,Flux CH4 [Tg/yr],Surf. conc. CH4 per flux [ppb yr/Tg],Flux H2/Flux CH4 [Tg H2/Tg CH4],ozone RF per flux [mW m-2 yr/ Tg],Strat H2O RF per flux [mW m-2 yr/ Tg]
anthro1,49.7,3.65,0.02,0.82,0.19
H2_avi,49.7,3.65,0.02,0.82,0.19
SSP119,37.8,3.78,0.06,0.89,0.22
SSP434,52.1,4.27,0.06,0.84,0.24
SSP585,58.6,4.17,0.06,0.85,0.23


## Table per flux H2 (including changes in methane)

In [165]:
#Reread - to get the other heading.
df_per_flux_h2_combined = pd.read_csv(outputpath + 'table_per_flux_h2.csv',index_col=0)
df_per_flux_h2_combined

,deltaH2,surf_h2_per_h2_flux,surf_ch4_per_h2_flux,ch4_flux_per_h2_flux,ch4_rf_per_h2_flux,ozone_rf_per_h2_flux,h2o_rf_per_h2_flux
CNTR,0.00,NaN,NaN,NaN,NaN,NaN,NaN
H2_avi,0.71,6.38,1.18,0.32,0.46,0.22,NaN
SSP119,9.90,6.36,1.12,0.30,0.43,0.22,0.17
SSP434,9.90,6.53,1.26,0.30,0.49,0.18,0.18
SSP585,9.89,6.42,1.33,0.32,0.51,0.18,0.17
antro1,0.95,6.26,1.12,0.31,0.43,0.22,NaN


In [166]:
df_per_flux_ch4_add  = pd.read_csv(outputpath + 'table_per_flux_ch4.csv',index_col=0)
df_per_flux_ch4_add

,deltaCH4,surf_ch4_per_ch4_flux,h2_flux_per_ch4_flux,ozone_rf_per_ch4_flux,h2o_rf_per_ch4_flux
CNTR,49.7,3.65,0.02,0.82,0.19
antro1,49.7,3.65,0.02,0.82,0.19
H2_avi,49.7,3.65,0.02,0.82,0.19
SSP119,37.8,3.78,0.06,0.89,0.22
SSP434,52.1,4.27,0.06,0.84,0.24
SSP585,58.6,4.17,0.06,0.85,0.23


In [167]:
frac = df_per_flux_h2_combined['ch4_flux_per_h2_flux'] #Tg CH4/Tg H2 /Tg CH4 = 1/Tg H2
frac

CNTR      NaN
H2_avi   0.32
SSP119   0.30
SSP434   0.30
SSP585   0.32
antro1   0.31
Name: ch4_flux_per_h2_flux, dtype: float64

In [168]:
df_per_flux_ch4_add
#Keep the following:
#deltaH2
#surf_h2_per_h2_flux keep as h2_flux_per_ch4_flux small
#surf_ch4_per_h2_flux
#ch4_flux_per_h2_flux
#ch4_rf_per_h2_flux

#add:
#trop_du_ozone_per_h2_flux
#strat_du_ozone_per_h2_flux
#ozone_rf_per_h2_flux
#h2o_rf_per_h2_flux
#aerosol_rf_per_h2_flux


df_per_flux_h2_combined['ozone_rf_per_h2_flux'] = df_per_flux_h2_combined['ozone_rf_per_h2_flux'] + df_per_flux_ch4_add['ozone_rf_per_ch4_flux']*frac
df_per_flux_h2_combined['h2o_rf_per_h2_flux'] = df_per_flux_h2_combined['h2o_rf_per_h2_flux'] + df_per_flux_ch4_add['h2o_rf_per_ch4_flux']*frac



#Save to file:
df_per_flux_h2_combined.to_csv(outputpath + 'table_per_flux_h2_combined.csv')


In [169]:
#Rename the columns:
columns_names={'deltaH2':'Flux H2 [Tg/yr]',
               'surf_h2_per_h2_flux': 'Surf. conc. H2 per flux [ppb yr/Tg]',
               'surf_ch4_per_h2_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'ch4_flux_per_h2_flux':'Flux CH4/Flux H2 [Tg CH4/Tg H2]',
               'ch4_rf_per_h2_flux':'CH4 RF per flux [mW m-2 yr/ Tg]',
               'trop_du_ozone_per_h2_flux':'Trop. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'strat_du_ozone_per_h2_flux':'Strat. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'ozone_rf_per_h2_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_h2_flux':'Strat. H2O RF per flux [mW m-2 yr/ Tg]',
               'aerosol_rf_per_h2_flux':'Aerosol RF per flux [mW m-2 yr/ Tg]'}
#Rename column names:
df_per_flux_h2_combined.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])


df_per_flux_h2_combined = df_per_flux_h2_combined.drop('CNTR')

sorted_list = ['antro1', 
               'H2_avi',
               'SSP119', 
               'SSP434', 
               'SSP585'] 
senslist = sorted_list
senslist = [sub.replace('antro', 'anthro') for sub in senslist]
df_per_flux_h2_combined = df_per_flux_h2_combined.reindex(sorted_list)


df_per_flux_h2_combined.to_csv(outputpath + 'table_per_flux_h2_combined_to_manuscript.csv')
df_per_flux_h2_combined

,Flux H2 [Tg/yr],Surf. conc. H2 per flux [ppb yr/Tg],Surf. conc. CH4 per flux [ppb yr/Tg],Flux CH4/Flux H2 [Tg CH4/Tg H2],CH4 RF per flux [mW m-2 yr/ Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat. H2O RF per flux [mW m-2 yr/ Tg]
antro1,0.95,6.26,1.12,0.31,0.43,0.47,NaN
H2_avi,0.71,6.38,1.18,0.32,0.46,0.49,NaN
SSP119,9.90,6.36,1.12,0.30,0.43,0.48,0.24
SSP434,9.90,6.53,1.26,0.30,0.49,0.43,0.25
SSP585,9.89,6.42,1.33,0.32,0.51,0.45,0.25


## H2 budget table

In [170]:
df_h2_burden

,CNTR,antro1,H2_avi
Scenario,,,
YR4,205,207,206
deltaH2,0.00,2.02,1.61


In [171]:
df_h2_surfconc

,CNTR,antro1,H2_avi
YR4,559,565,564
delta,0.00,5.92,4.56


In [172]:
df_budget_h2 = pd.concat([df_h2_burden.iloc[0],
                          df_h2_surfconc.iloc[0],
                          df_h2_atmloss,
                          df_h2_atmprod,
                          df_h2_drydep,
                          df_h2_estemis, 
                          df_h2_atm_lifetime,
                          df_h2_soil_sink_lifetime,
                          df_h2_lifetime
                         ],axis=1)

df_budget_h2.columns = ['H2 burden [Tg]',
                        'H2 surf. conc [ppbv]',
                        'H2 atm loss [Tg/yr]',
                        'H2 atm prod [Tg/yr]',
                        'H2 soil sink [Tg/yr]',
                        'H2 estimated emissions [Tg/yr]',
                        'H2 atm lifetime [yrs]',
                        'H2 soil sink lifetime [yrs]',
                        'H2 total lifetime [yrs]',]
df_budget_h2 = df_budget_h2.sort_index()

df_budget_h2



,H2 burden [Tg],H2 surf. conc [ppbv],H2 atm loss [Tg/yr],H2 atm prod [Tg/yr],H2 soil sink [Tg/yr],H2 estimated emissions [Tg/yr],H2 atm lifetime [yrs],H2 soil sink lifetime [yrs],H2 total lifetime [yrs]
CNTR,205,559,29.2,55.8,58.0,31.4,7.02,3.53,2.35
H2_avi,206,564,29.4,55.6,58.5,32.3,7.02,3.52,2.35
antro1,207,565,29.4,55.8,58.7,32.3,7.02,3.52,2.34


In [173]:
df_budget_h2_ssp = pd.concat([df_h2_burden_ssp.iloc[0],
                              df_h2_surfconc_ssp.iloc[0],
                              df_h2_atmloss_ssp,
                              df_h2_atmprod_ssp,
                              df_h2_drydep_ssp,
                              df_h2_estemis_ssp, 
                              df_h2_atm_lifetime_ssp,
                              df_h2_soil_sink_lifetime_ssp,
                              df_h2_lifetime_ssp
                             ],axis=1)

df_budget_h2_ssp.columns = ['H2 burden [Tg]',
                            'H2 surf. conc [ppbv]',
                        'H2 atm loss [Tg/yr]',
                        'H2 atm prod [Tg/yr]',
                        'H2 soil sink [Tg/yr]',
                        'H2 estimated emissions [Tg/yr]',
                        'H2 atm lifetime [yrs]',
                        'H2 soil sink lifetime [yrs]',
                        'H2 total lifetime [yrs]',]
df_budget_h2_ssp = df_budget_h2_ssp.sort_index()
df_budget_h2_ssp

,H2 burden [Tg],H2 surf. conc [ppbv],H2 atm loss [Tg/yr],H2 atm prod [Tg/yr],H2 soil sink [Tg/yr],H2 estimated emissions [Tg/yr],H2 atm lifetime [yrs],H2 soil sink lifetime [yrs],H2 total lifetime [yrs]
10CH4_SSP119,188,514,25.2,47.2,53.4,31.4,7.48,3.52,2.39
10CH4_SSP434,235,640,28.6,63.4,66.1,31.3,8.21,3.55,2.48
10CH4_SSP585,245,668,31.4,69.0,68.9,31.3,7.81,3.56,2.44
CNTR_SSP119,181,495,24.9,45.0,51.5,31.4,7.26,3.51,2.37
CNTR_SSP434,225,613,28.3,60.4,63.5,31.3,7.95,3.54,2.45
CNTR_SSP585,234,638,31.0,65.6,66.0,31.3,7.55,3.55,2.41
antro10_SSP119,203,558,27.8,44.9,58.5,41.4,7.29,3.46,2.35
antro10_SSP434,247,678,31.0,60.3,70.7,41.3,7.98,3.50,2.43
antro10_SSP585,256,702,33.8,65.5,73.0,41.3,7.57,3.51,2.40


In [174]:
df_budget_h2 = pd.concat([df_budget_h2,df_budget_h2_ssp],sort=False)
df_budget_h2

,H2 burden [Tg],H2 surf. conc [ppbv],H2 atm loss [Tg/yr],H2 atm prod [Tg/yr],H2 soil sink [Tg/yr],H2 estimated emissions [Tg/yr],H2 atm lifetime [yrs],H2 soil sink lifetime [yrs],H2 total lifetime [yrs]
CNTR,205,559,29.2,55.8,58.0,31.4,7.02,3.53,2.35
H2_avi,206,564,29.4,55.6,58.5,32.3,7.02,3.52,2.35
antro1,207,565,29.4,55.8,58.7,32.3,7.02,3.52,2.34
10CH4_SSP119,188,514,25.2,47.2,53.4,31.4,7.48,3.52,2.39
10CH4_SSP434,235,640,28.6,63.4,66.1,31.3,8.21,3.55,2.48
10CH4_SSP585,245,668,31.4,69.0,68.9,31.3,7.81,3.56,2.44
CNTR_SSP119,181,495,24.9,45.0,51.5,31.4,7.26,3.51,2.37
CNTR_SSP434,225,613,28.3,60.4,63.5,31.3,7.95,3.54,2.45
CNTR_SSP585,234,638,31.0,65.6,66.0,31.3,7.55,3.55,2.41
antro10_SSP119,203,558,27.8,44.9,58.5,41.4,7.29,3.46,2.35


In [175]:
df_budget_h2.loc[['CNTR','CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']]

,H2 burden [Tg],H2 surf. conc [ppbv],H2 atm loss [Tg/yr],H2 atm prod [Tg/yr],H2 soil sink [Tg/yr],H2 estimated emissions [Tg/yr],H2 atm lifetime [yrs],H2 soil sink lifetime [yrs],H2 total lifetime [yrs]
CNTR,205,559,29.2,55.8,58.0,31.4,7.02,3.53,2.35
CNTR_SSP119,181,495,24.9,45.0,51.5,31.4,7.26,3.51,2.37
CNTR_SSP434,225,613,28.3,60.4,63.5,31.3,7.95,3.54,2.45
CNTR_SSP585,234,638,31.0,65.6,66.0,31.3,7.55,3.55,2.41


In [176]:
df_budget_h2.to_csv(outputpath + 'table_budget_h2.csv')

## CH4 budget table

In [177]:
df_ch4_loss_other_strat_conc.loc['CTRL']

np.float64(20.72921738808889)

In [178]:
budget_ch4 = np.array([df_ch4_burden_conc.loc['CTRL'],
                           df_ch4_surfconc_conc.loc['CTRL'],
                           df_ch4_loss_conc.loc['CTRL'],
                           df_ch4_loss_other_strat_conc.loc['CTRL'],
                           df_ch4_loss_soil_conc.loc['CTRL'],
                           df_ch4_loss_conc.loc['CTRL']+df_ch4_loss_other_strat_conc.loc['CTRL']+df_ch4_loss_soil_conc.loc['CTRL'],
                           df_ch4_lifetime_conc.loc['CTRL'],
                           df_ch4_tot_lifetime_conc.loc['CTRL']
                           ])

budget_ch4_columns = ['CH4 burden [Tg]','CH4 surface conc. [ppbv]',
                         'CH4 chem loss OH [Tg/yr]', 'CH4 chem loss other strat [Tg/yr]','CH4 loss soil [Tg/yr]',
                         'CH4 total loss [Tg/yr]',
                         'CH4 lifetime due to OH (whole atmosphere) [yrs]','Total CH4 lifetime [yrs]'] 

df_budget_ch4 = pd.DataFrame(data=budget_ch4,index=budget_ch4_columns,columns=['CTRL'])


In [179]:
temp =df_ch4_loss_ssp.loc[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']]+df_ch4_loss_other_strat_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']]+df_ch4_loss_soil_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']]
temp.iloc[0]

CNTR_SSP119   552
CNTR_SSP434   787
CNTR_SSP585   910
Name: YR13, dtype: float64

In [180]:
df_ch4_burden_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']]

,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585
Scenario,,,
YR13,"3,916","6,104","6,713"
deltaCH4,0.00,0.00,0.00


In [181]:
budget_ch4_ssp = np.array([df_ch4_burden_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']].iloc[0].values,
                           df_ch4_surfconc_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']].iloc[0].values,
                           df_ch4_loss_ssp.loc[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']].values,
                           df_ch4_loss_other_strat_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']].iloc[0].values,
                           df_ch4_loss_soil_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']].iloc[0].values,
                           temp.iloc[0].values,
                           df_ch4_lifetime_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']].iloc[0].values,
                           df_ch4_tot_lifetime_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']].iloc[0].values
                           ])

budget_ch4_columns = ['CH4 burden [Tg]','CH4 surface conc. [ppbv]',
                        'CH4 chem loss OH [Tg/yr]', 'CH4 chem loss other strat [Tg/yr]','CH4 loss soil [Tg/yr]',
                         'CH4 total loss [Tg/yr]',
                         'CH4 lifetime due to OH (whole atmosphere) [yrs]','Total CH4 lifetime [yrs]'] 

df_budget_ch4_ssp = pd.DataFrame(data=budget_ch4_ssp,index=budget_ch4_columns,columns=['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585'])
df_budget_ch4_ssp.T

,CH4 burden [Tg],CH4 surface conc. [ppbv],CH4 chem loss OH [Tg/yr],CH4 chem loss other strat [Tg/yr],CH4 loss soil [Tg/yr],CH4 total loss [Tg/yr],CH4 lifetime due to OH (whole atmosphere) [yrs],Total CH4 lifetime [yrs]
CNTR_SSP119,"3,916","1,427",511,16.3,24.5,552,7.67,7.10
CNTR_SSP434,"6,104","2,223",724,25.4,38.2,787,8.43,7.75
CNTR_SSP585,"6,713","2,446",841,28.0,42.0,910,7.99,7.37


In [182]:
df_budget_ch4 = pd.concat([df_budget_ch4,df_budget_ch4_ssp],axis=1,sort=False)
df_budget_ch4.T.to_csv(outputpath + 'table_budget_ch4.csv')

df_budget_ch4.T

,CH4 burden [Tg],CH4 surface conc. [ppbv],CH4 chem loss OH [Tg/yr],CH4 chem loss other strat [Tg/yr],CH4 loss soil [Tg/yr],CH4 total loss [Tg/yr],CH4 lifetime due to OH (whole atmosphere) [yrs],Total CH4 lifetime [yrs]
CTRL,"4,975","1,813",674,20.7,31.1,726,7.38,6.85
CNTR_SSP119,"3,916","1,427",511,16.3,24.5,552,7.67,7.10
CNTR_SSP434,"6,104","2,223",724,25.4,38.2,787,8.43,7.75
CNTR_SSP585,"6,713","2,446",841,28.0,42.0,910,7.99,7.37


In [183]:
#Write AGWP values to file
df_h2_agwp  = pd.concat([df_h2_agwp_ch4,
                         df_h2_agwp_o3,
                         df_h2_agwp_h2o,
                         df_h2_agwp_ch4ind_o3,
                         df_h2_agwp_ch4ind_h2o],axis=1,sort=False)

df_h2_agwp.to_csv(outputpath + 'table_h2_agwp.csv') 
df_h2_agwp

,h2_agwp_ch4,h2_agwp_o3,0,h2_agwp_ch4ind_o3,h2_agwp_ch4ind_h2o
CNTR,NaN,NaN,NaN,NaN,NaN
antro1,0.43,0.22,NaN,0.25,0.06
H2_avi,0.46,0.22,NaN,0.27,0.06


# Appendix:

## Methane feedback factor:

### Atmospheric mass conversion CH4  [Tg/ppb] (from perturbations)

In [184]:
df_ch4_burden_per_conc  = df_ch4_burden_conc.loc['deltaCH4']/df_ch4_surfconc_conc.loc['deltaCH4']
df_ch4_burden_per_conc

np.float64(2.7494761488687973)

In [185]:
df_ch4_surfconc_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
YR13,"1,570","2,445","2,690","1,427","2,223","2,446","1,427","2,223","2,446"
delta,143,222,245,0.00,0.00,0.00,0.00,0.00,0.00


In [186]:
df_ch4_burden_per_conc_ssp  = df_ch4_burden_ssp.loc['deltaCH4']/df_ch4_surfconc_ssp.loc['delta']
df_ch4_burden_per_conc_ssp

10CH4_SSP119        2.75
10CH4_SSP434        2.75
10CH4_SSP585        2.75
CNTR_SSP119          NaN
CNTR_SSP434          NaN
CNTR_SSP585          NaN
antro10_SSP119   189,221
antro10_SSP434    33,766
antro10_SSP585    34,326
dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

In [187]:
df_w_o_feedback =df_ch4_tot_lifetime_conc.loc['CTRL']/df_ch4_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback_to_file = pd.DataFrame(data=[df_w_o_feedback],index=['OSLOCTM3'])
df_w_o_feedback_to_file.to_csv(outputpath + 'increase_w_o_feedback.csv')
df_w_o_feedback

np.float64(2.492397459152925)

In [188]:
df_w_o_feedback_ssp =df_ch4_tot_lifetime_ssp/df_ch4_burden_per_conc_ssp #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback_ssp[['10CH4_SSP119','10CH4_SSP434','10CH4_SSP585']]

df_w_o_feedback_to_file_ssp = pd.DataFrame(data=[df_w_o_feedback_ssp.iloc[0][['10CH4_SSP119','10CH4_SSP434','10CH4_SSP585']]],index=['OSLOCTM3'])
df_w_o_feedback_to_file_ssp

df_w_o_feedback_ssp = df_w_o_feedback_ssp[['10CH4_SSP119','10CH4_SSP434','10CH4_SSP585']]
df_w_o_feedback_ssp.columns = ['SSP119','SSP434','SSP585']
df_w_o_feedback_to_file_ssp.to_csv(outputpath + 'increase_w_o_feedback_ssp.csv')
df_w_o_feedback_ssp

,SSP119,SSP434,SSP585
Scenario,,,
YR13,2.66,2.91,2.77


### Feedback factor: increase CH4 with feedback/ increase CH4 without feedback

In [189]:
df_feedback_factor_ch4 = df_surf_ch4_per_ch4_flux_conc/df_w_o_feedback

df_feedback_to_file = pd.DataFrame(data=[df_feedback_factor_ch4],index=['CTRL'])

df_feedback_factor_ch4

np.float64(1.4645748747885994)

In [190]:
df_feedback_factor_ch4_ssp = df_surf_ch4_per_ch4_flux_ssp/df_w_o_feedback
df_feedback_to_file_ssp = pd.DataFrame(data=[df_feedback_factor_ch4_ssp],index=['CTRL'])
df_feedback_factor_ch4_ssp


SSP119   1.52
SSP434   1.71
SSP585   1.67
Name: surf_ch4_per_ch4_flux, dtype: object

In [191]:
df_feedback_factor_ch4_ssp.index

Index(['SSP119', 'SSP434', 'SSP585'], dtype='object')

In [192]:
df_feedback_factor_ch4_comb =  pd.DataFrame(data=[],index=['OSLOCTM'],columns=df_h2_gwp.columns)
df_feedback_factor_ch4_comb[['antro1','H2_avi']]=df_feedback_factor_ch4

scen_list = ['SSP119', 'SSP434', 'SSP585']
for scen in scen_list:
    df_feedback_factor_ch4_comb[scen]=df_feedback_factor_ch4_ssp[scen]

df_feedback_factor_ch4_comb.to_csv(outputpath + 'feedback_factor_ch4.csv')
df_feedback_factor_ch4_comb

,antro1,H2_avi,SSP119,SSP434,SSP585
OSLOCTM,1.46,1.46,1.52,1.71,1.67


Split the CH4 GWP into direct and indirect based on the feedback factor.

In [193]:
df_feedback_frac = 1.0 - (1.0/df_feedback_factor_ch4_comb)
df_feedback_frac

,antro1,H2_avi,SSP119,SSP434,SSP585
OSLOCTM,0.32,0.32,0.34,0.42,0.40


In [194]:
#Save to file:
df_h2_gwp.loc['CH4dir'] = df_h2_gwp.loc['CH4']*(1.0-df_feedback_frac.loc['OSLOCTM'])
df_h2_gwp.loc['CH4indir'] = df_h2_gwp.loc['CH4']*df_feedback_frac.loc['OSLOCTM']

df_h2_gwp = df_h2_gwp.drop(['total','CH4'])
df_h2_gwp.to_csv(outputpath + 'table_h2_gwp.csv')

df_h2_gwp

,antro1,H2_avi,SSP119,SSP434,SSP585
O3,2.41,2.51,2.48,2.05,2.01
strat H2O,NaN,NaN,1.95,1.96,1.92
O3 CH4ind,2.82,2.98,2.94,2.79,3.03
strat H2O CH4ind,0.66,0.70,0.73,0.79,0.83
CH4dir,3.30,3.48,3.18,3.18,3.43
CH4indir,1.53,1.62,1.64,2.26,2.31


In [195]:
df_h2_gwp.loc['CH4indir']/(df_h2_gwp.loc['CH4dir']+df_h2_gwp.loc['CH4indir'])

antro1   0.32
H2_avi   0.32
SSP119   0.34
SSP434   0.42
SSP585   0.40
dtype: float64

In [196]:
df_feedback_factor_ch4

np.float64(1.4645748747885994)

In [197]:
df_feedback_factor_ch4_ssp

SSP119   1.52
SSP434   1.71
SSP585   1.67
Name: surf_ch4_per_ch4_flux, dtype: object

## Hydrogen feedback factor:

### Atmospheric mass conversion H2  [Tg/ppb] (from perturbations)

In [198]:
df_h2_burden_per_conc  = df_h2_burden.loc['deltaH2']/df_h2_surfconc.loc['delta']
df_h2_burden_per_conc.name = 'h2_burden_per_conc'
df_h2_burden_per_conc

CNTR      NaN
antro1   0.34
H2_avi   0.35
Name: h2_burden_per_conc, dtype: float64

In [199]:
df_h2_burden_per_conc_ssp  = df_h2_burden_ssp.loc['deltaH2']/df_h2_surfconc_ssp.loc['delta']
df_h2_burden_per_conc_ssp.name = 'h2_burden_per_conc'
df_h2_burden_per_conc_ssp

10CH4_SSP119     0.37
10CH4_SSP434     0.38
10CH4_SSP585     0.38
CNTR_SSP119       NaN
CNTR_SSP434       NaN
CNTR_SSP585       NaN
antro10_SSP119   0.35
antro10_SSP434   0.35
antro10_SSP585   0.35
Name: h2_burden_per_conc, dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

In [200]:
df_w_o_feedback_h2 =df_h2_lifetime.loc['CNTR']/df_h2_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback_h2

CNTR      NaN
antro1   6.87
H2_avi   6.66
Name: h2_burden_per_conc, dtype: float64

In [201]:
df_w_o_feedback_h2_ssp =df_h2_lifetime_ssp/df_h2_burden_per_conc_ssp #Lifetime [yr] / [Tg/ppb] 

scen_dict = {'antro10_SSP119':'SSP119',
             'antro10_SSP434':'SSP434',
             'antro10_SSP585':'SSP585'}

df_w_o_feedback_h2_ssp =df_w_o_feedback_h2_ssp[['antro10_SSP119','antro10_SSP434','antro10_SSP585']]

df_w_o_feedback_h2_ssp.index = ['SSP119','SSP434','SSP585']


df_w_o_feedback_h2_ssp

SSP119   6.78
SSP434   7.02
SSP585   6.92
dtype: float64

### Feedback factor: increase H2 with feedback/ increase H2 without feedback

If you take the control lifetime from the budget terms and divide it into the lifetime of the perturbation (the added burden from the 10% increase divided by the change in chemical flux from that perturbation.) You should get the feed back factor.

In [202]:
df_feedback_factor_h2 = df_surf_h2_per_h2_flux/df_w_o_feedback_h2
df_feedback_factor_h2.name = 'feedback_factor_h2'
df_feedback_factor_h2

CNTR      NaN
antro1   0.91
H2_avi   0.96
Name: feedback_factor_h2, dtype: float64

In [203]:
df_surf_h2_per_h2_flux_ssp_v2 = df_surf_h2_per_h2_flux_ssp[['antro10_SSP119','antro10_SSP434','antro10_SSP585']]
df_surf_h2_per_h2_flux_ssp_v2.index = ['SSP119','SSP434','SSP585']
df_surf_h2_per_h2_flux_ssp_v2

SSP119   6.36
SSP434   6.53
SSP585   6.42
Name: surf_h2_per_h2_flux, dtype: float64

In [204]:
df_feedback_factor_h2_ssp = df_surf_h2_per_h2_flux_ssp_v2/df_w_o_feedback_h2_ssp

df_feedback_factor_h2_ssp

SSP119   0.94
SSP434   0.93
SSP585   0.93
dtype: float64

In [205]:
df_feedback_factor_h2_comb =  pd.DataFrame(data=[],index=['OSLOCTM'],columns=df_h2_gwp.columns)
for scen in df_feedback_factor_h2.drop('CNTR').index:
    df_feedback_factor_h2_comb[scen] = df_feedback_factor_h2[scen]
df_feedback_factor_h2_comb

,antro1,H2_avi,SSP119,SSP434,SSP585
OSLOCTM,0.91,0.96,NaN,NaN,NaN


In [206]:
scen_list = ['SSP119', 'SSP434', 'SSP585']
for scen in scen_list:
    df_feedback_factor_h2_comb[scen]=df_feedback_factor_h2_ssp[scen]

df_feedback_factor_h2_comb.to_csv(outputpath + 'feedback_factor_h2.csv')
df_feedback_factor_h2_comb

,antro1,H2_avi,SSP119,SSP434,SSP585
OSLOCTM,0.91,0.96,0.94,0.93,0.93


### Change in lifetime per flux

In [207]:
df_ch4_lifetime.loc['deltaH2'] = df_ch4_lifetime.iloc[0]-df_ch4_lifetime['CNTR'].iloc[0]
df_ch4_lifetime.loc['deltaCH4'] = df_ch4_lifetime_conc.loc['10CH4']-df_ch4_lifetime_conc.loc['CTRL']
df_ch4_lifetime

,CNTR,antro1,H2_avi
Scenario,,,
YR4,7.39,7.40,7.40
deltaH2,0.00,0.00,0.00
deltaCH4,0.24,0.24,0.24


In [208]:
#Direct (changes in methane lifetime per h2 flux [days per Tg H2])
df_ch4_lifetime_per_h2_flux =  df_ch4_lifetime.loc['deltaH2']/df_h2_flux['deltaH2']
df_ch4_lifetime_per_h2_flux*365.0 #Days

CNTR      NaN
antro1   1.24
H2_avi   1.31
Name: deltaH2, dtype: float64

In [209]:
#Indirect (changes in methane lifetime per h2 flux [days per Tg H2] due to changes in methane):
df_ch4_lifetime_per_ch4_flux =  df_ch4_lifetime.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_ch4_lifetime_per_ch4_flux*365.0*df_ch4_flux_per_h2_flux


CNTR      NaN
antro1   0.54
H2_avi   0.57
dtype: float64

### Check that delta flux equal delta burden divided by lifetime including the feedback effect

In [210]:
df_feedback_factor_h2

CNTR      NaN
antro1   0.91
H2_avi   0.96
Name: feedback_factor_h2, dtype: float64

In [211]:
df_h2_burden

,CNTR,antro1,H2_avi
Scenario,,,
YR4,205,207,206
deltaH2,0.00,2.02,1.61


In [212]:
#Delta burden divided by tau*ff
test = df_h2_burden.loc['deltaH2']/(df_h2_lifetime.loc['CNTR']*df_feedback_factor_h2)
#print(test)
#print(df_h2_flux['deltaH2'])
test - df_h2_flux['deltaH2']

CNTR      NaN
antro1   0.00
H2_avi   0.00
dtype: float64

## Feedback factor summary

The feedback factor calculation employed above is equivalent with a division between the perturbation lifetime and the total lifetime, where the perturbation lifetime is defined as the burden change diveded by the flux change in the perturbed simulation relative to the control simulation

In [213]:
print(df_feedback_factor_ch4)
test_ff =  (df_ch4_burden_conc.loc['deltaCH4']/df_ch4_flux_conc.loc['deltaCH4'])/df_ch4_tot_lifetime_conc.loc['CTRL']

test_ff

1.4645748747885994


np.float64(1.4645748747885996)

In [214]:
#print(df_feedback_factor_h2)
test_ff =  (df_h2_burden.loc['deltaH2']/df_h2_flux['deltaH2'])/df_h2_lifetime.loc['CNTR']

test_ff - df_feedback_factor_h2



CNTR       NaN
antro1   -0.00
H2_avi    0.00
dtype: float64

## Calculating values to be used for adding composition changes from the methan run

How to combine composition changes in the methane run and the hydrogen run. Fraction calculated based on the 10 Tg yr-1 increase in anthropogenic H2 emissions

In [215]:
df_per_flux_h2.loc['anthro1']

Flux H2 [Tg/yr]                          0.95
Surf. conc. H2 per flux [ppb yr/Tg]      6.26
Surf. conc. CH4 per flux [ppb yr/Tg]     1.12
Flux CH4/Flux H2 [Tg CH4/Tg H2]          0.31
CH4 RF per flux [mW m-2 yr/ Tg]          0.43
ozone RF per flux [mW m-2 yr/ Tg]        0.22
Strat. H2O RF per flux [mW m-2 yr/ Tg]    NaN
Name: anthro1, dtype: float64

In [216]:
df_per_flux_ch4.loc['anthro1']

Flux CH4 [Tg/yr]                        49.7
Surf. conc. CH4 per flux [ppb yr/Tg]    3.65
Flux H2/Flux CH4 [Tg H2/Tg CH4]         0.02
ozone RF per flux [mW m-2 yr/ Tg]       0.82
Strat H2O RF per flux [mW m-2 yr/ Tg]   0.19
Name: anthro1, dtype: object

This is based on the 10% increase in surface methane concentration

Fraction of changes in atmospheric composition to be added to the changes in the pure hydrogen perturbation. To include methane induced changes.

  [Tg CH4/Tg H2]*[Tg H2/yr]/[Tg CH4/yr]

In [218]:
frac = df_per_flux_h2['Flux CH4/Flux H2 [Tg CH4/Tg H2]'].loc['anthro1']*df_per_flux_h2['Flux H2 [Tg/yr]'].loc['anthro1']/df_per_flux_ch4['Flux CH4 [Tg/yr]'].loc['anthro1']
frac

np.float64(0.005854534316334419)